# Scaling a value investor’s stock selection using machine learning
## Thesis by Selina Waber, November 2023

This code is extracted from my thesis, with all confidential information carefully excluded. Recognizing the sensitivity of financial data, I have omitted any references to such details. As a result, many functions may not execute properly after removal. This repository is intended solely for reading purposes, providing an anonymized version to showcase  my Python skills.









# Imports


In [ ]:
import os 
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import matplotlib.gridspec as gridspec
import scipy.stats as stats
from sklearn import metrics
from sklearn.base import clone
import shap

from mpl_toolkits.mplot3d import Axes3D

from scipy.stats import chi2_contingency, f_oneway
from scipy.stats import zscore

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller


from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit

from sklearn.inspection import permutation_importance

from sklearn.discriminant_analysis import StandardScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve

from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SMOTENC

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import plot_tree

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc
from sklearn.metrics import average_precision_score, precision_recall_curve



from imblearn.pipeline import Pipeline as imbpipeline

import xgboost as xgb
from xgboost import XGBClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from scipy.stats import loguniform

from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

from catboost import CatBoostClassifier


## not used?
from sklearn.calibration import LabelEncoder
from sklearn.manifold import TSNE
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
############## Ignore Filterwarnings ##############
# to avoid "UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples." spam
# for models where some element(s) in the confusion matrix is/are 0
import warnings
warnings.filterwarnings("ignore") 

# OurDataset()

The dataset was provided by the company as an .xlsx Excel file, listing all the stock sales and buys over a given period of time while listing 15 financial indicators. 

We define a class `OurDataset()` to handle all stages of pre-processing and missing data imputation, outlier cleaning and train/validation/test split. 



In [ ]:
class OurDataset():
    def __init__(self, path=None):
        self.X: pd.DataFrame = None
        self.y: pd.DataFrame = None
        self.data: pd.DataFrame = None
        if path is not None:
            self.load_dataset(path)                  
        self.X_train: pd.DataFrame = None
        self.X_test: pd.DataFrame = None
        self.y_train: pd.DataFrame = None
        self.y_test: pd.DataFrame = None
        self.X_val: pd.DataFrame = None
        self.y_val: pd.DataFrame = None

        self.X_train_not_imputed = None
        self.X_val_not_imputed = None
        self.X_test_not_imputed = None

    def set_data(self, dataset):
        self.data = dataset
        self.y = self.data['Target'] 
        self.X = self.data.drop( ['Date',  'Company Name', 'Bloomberg Ticker',
                             'ISIN', 'CIQ ISIN', 'Acquired', 'Target'], axis = 1)
        

    def load_dataset(self, project_directory):
        data_directory = os.path.join(project_directory, 'Data')
        ## Have to do some formating, since column names/ header starts at second row
        xls = pd.ExcelFile(os.path.join(data_directory, f'data.xlsx'))
        df_bought = pd.read_excel(xls, 'Käufe', skiprows=1)               
        df_sold = pd.read_excel(xls, 'Verk', skiprows=1)
   
        
        ### Add the Column 'Acquired' and creating the Target class buy=1 and sell=0.
        df_bought['Acquired']= 'buy' ## Adding Columns
        df_bought['Target'] = 1

        df_sold['Acquired'] = 'sell' ## Adding Columns
        df_sold['Target'] = 0

        # Merging the two datasets df_sold and df_bought together.
        df = pd.concat([df_bought, df_sold], axis=0, ignore_index=True) # Concat the two datasets

        df['Acquired'] = df['Acquired'].astype('category') # Converting type of columns to category
        df['Target'] = df['Target'].astype('category') ## convert 'Target' to categorical data
        

        # Convert from object to float, and convert non numbers to NA in floats
        list_original_indicators = ['Financial_Ratio_1','Financial_Ratio_2', 'Financial_Ratio_3', 'Financial_Ratio_4',
       'Financial_Ratio_5', 'Financial_Ratio_6', 'Financial_Ratio_7', 'Financial_Ratio_8',
       'Financial_Ratio_9', 'Financial_Ratio_10',
       'Financial_Ratio_11', 'Financial_Ratio_12',
       'Financial_Ratio_13', 'Financial_Ratio_14']

        for col in list_original_indicators:
            df[col] = pd.to_numeric(df[col], errors='coerce') #If ‘coerce’, then invalid parsing will be set as NaN.
       
        self.set_data(df) #important, this set always adjusts X and y as wells as data!!

        ## Print the Shape to check
        print("Shape Original Dataset:", df.shape)
        print("========================================")
        

    def remove_timestamps(self):
        # Dropping the columns which where only used by the client for downloading from Bloomberg
        self.set_data( self.data.drop(columns=['Datum -5J', 'Datum -4J', 'Datum -3J', 'Datum -2J', 'Datum -1J', 'Datum -6m', 'Datum -3m', 'Datum -1m' ]))
        print("Removing Bloomberg Timestamps, new shape is:", self.data.shape)
        print("========================================")

    def remove_duplicates(self):
        ## Dropping Row Duplicates since there is no information in which quantities stock were bought/sold
        # Select duplicate row based on the indicator columns and target columns, but not on Date! or on the company name since the company name is not always written the same!
        subset1 = [ 'Bloomberg Ticker', 'ISIN', 'CIQ ISIN','Financial_Ratio_1','Financial_Ratio_2', 'Financial_Ratio_3', 'Financial_Ratio_3',
       'Financial_Ratio_4', 'Financial_Ratio_6', 'Financial_Ratio_7', 'Financial_Ratio_8',
       'Financial_Ratio_9', 'Financial_Ratio_10',
       'Financial_Ratio_11', 'Financial_Ratio_12',
       'Financial_Ratio_13', 'Financial_Ratio_14', 'Acquired', 'Target']
        
        df_duplicates = self.data[self.data.duplicated(keep=False, subset=subset1)] # keep = False marks all duplicates as Ture
        print("There are this many row duplicates:", df_duplicates.shape[0])
        #print(df_duplicates[['Date', 'Company Name']])
        df_duplicates.to_excel("duplicates.xlsx", index=True) 
        #### Should I adjust the index???
        self.set_data(self.data.drop_duplicates(subset=subset1, keep="first"))
        print("Removing Duplicated Rows, new Shape is:", self.data.shape)
        print("========================================")

    def sort_after_date(self):
        ## Dates are formated in a mess.....
        for i, dateval in enumerate(self.data.Date):
            try:
                pd.to_datetime(dateval, format='mixed', dayfirst=True)
            except:
                #print(dateval)
                new_dateval = str(dateval).replace(',', '.').replace('..', '.') # attempt to correct the format in a loop
                self.data.replace(to_replace=dateval, value=new_dateval, inplace=True)

        # Format Dates manually...
                     

        # Finally convert to datetime and sort the dataset!
        self.data['Date'] = pd.to_datetime(self.data['Date'], format="mixed", dayfirst= True)# convert to datetime 

        # Add unique seconds within each group of identical dates
        #self.data['Date'] = self.data.groupby('Date').cumcount().apply(lambda x: pd.to_timedelta(x, unit='s')) + self.data['Date']

        ## Finally sorting the dataframe
        self.set_data(self.data.sort_values(by='Date').reset_index(drop=True)) # df gets sorted in ascending order of dates, Reset the Index here!!
        print("========================================")
        print("Dataset sorted after Date, Index is resetted")
        print("========================================")
        self.data.to_excel("sorted_output.xlsx")

    def split_dataset(self, imputer, remove_outliers, shuffle):
        if not imputer in ["drop_all", "mean", "median", "KNN", "iterative", "keep_missing", None]:
            raise Exception(f"Invalid imputer type `{imputer}`")
        
        ## Train-Test Split
        if shuffle == False:
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, stratify = None, test_size = 0.2, shuffle=False, random_state = 42) 
            self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(self.X_train, self.y_train, stratify= None, test_size = 0.2, shuffle =False, random_state = 42) 
            print("Train,Validation and Test Split done") 
            self.plot_missing_data(self.X_train, self.X_val, self.X_test)
            self.plot_class_distribution(self.y_train, self.y_val, self.y_test)
            self.print_start_end_time(self.y_train, self.y_val, self.y_test)
            ## Can I do this like that?
            self.X_train_not_imputed = self.X_train
            self.X_val_not_imputed = self.X_val
            self.X_test_not_imputed = self.X_test

        elif shuffle == True:
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y,  test_size = 0.2, shuffle=True, stratify = self.y, random_state = 42) 
            self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(self.X_train, self.y_train, test_size = 0.2, shuffle =True, stratify = self.y_train, random_state = 42) 
            print("Train,Validation and Test Split done") 
            self.plot_missing_data(self.X_train, self.X_val, self.X_test)
            self.plot_class_distribution(self.y_train, self.y_val, self.y_test)
            print("========================================")
            ## Can I do this like that?
            self.X_train_not_imputed = self.X_train
            self.X_val_not_imputed = self.X_val
            self.X_test_not_imputed = self.X_test

        else:
            raise Exception(f"Invalid Shuffle `{shuffle}`")


        if imputer != None:
            self.impute_missing_data(imputer)
            #In order to remove outliers with IsolationForest, missing values have to be imputed or dropped
            if remove_outliers: # maybe it would make more sense to first remove outliers then impute but IsolationForest does not accecpt NAN!
                if imputer=="keep_missing":
                    raise Exception(f"Removing outliers is incompatible with option `{imputer}`")
                self.remove_outliers()
        
    
    def remove_outliers(self):
        # Isolation Forest
        outliers = IsolationForest(random_state = 42, contamination = 0.05).fit(self.X_train) # fit Isolation Forest only to training data
        inliers_train = outliers.predict(self.X_train)
        inliers_test = outliers.predict(self.X_test)
        inliers_val = outliers.predict(self.X_val)

        # Remove outliers where 1 represent inliers and -1 represent outliers:
        self.X_train = self.X_train[np.where(inliers_train == 1, True, False)]
        self.y_train = self.y_train[np.where(inliers_train == 1, True, False)]
        self.X_test = self.X_test[np.where(inliers_test == 1, True, False)]
        self.y_test = self.y_test[np.where(inliers_test == 1, True, False)]
        self.X_val = self.X_val[np.where(inliers_val == 1, True, False)]
        self.y_val = self.y_val[np.where(inliers_val == 1, True, False)]

        print("========================================")
        print("Outliers removed:")
        print(f"Shape of X_train is {self.X_train.shape}, and y_train is: {len(self.y_train)}")
        print(f"Shape of X_val is: {self.X_val.shape} and y_val is: {len(self.y_val)}")
        print(f"Shape of X_test is:{self.X_test.shape} and y_test is: {len(self.y_test)}")
        print("========================================")
 

    def impute_missing_data(self, imputer):

        if imputer == "drop_all":
            # Drop missing values from X_train and adjust y_train
            self.X_train = self.X_train.dropna()
            self.y_train = self.y_train[self.X_train.index]
            # Drop missing values from X_test and adjust y_test
            self.X_test = self.X_test.dropna()
            self.y_test = self.y_test[self.X_test.index]
            # Drop missing values from X_val and adjust y_val
            self.X_val = self.X_val.dropna()
            self.y_val = self.y_val[self.X_val.index]

        else:
            df_NA_train = self.X_train.copy(deep=True)
            df_NA_val = self.X_val.copy(deep=True)
            df_NA_test = self.X_test.copy(deep=True)
            
            for col in self.X.columns.values:
                df_NA_train[f'{col}_NA'] = self.X_train[col].isna().astype("category")  # convert to category
                df_NA_test[f'{col}_NA'] = self.X_test[col].isna().astype("category")  # convert to category
                df_NA_val[f'{col}_NA'] = self.X_val[col].isna().astype("category")  # convert to category

            if imputer =="keep_missing":
                self.X_train = self.X_train
                self.y_train = self.y_train
                self.y_val = self.y_val

            elif imputer == "median":
                imputer = SimpleImputer(strategy='median')
                self.X_train = pd.DataFrame(imputer.fit_transform(self.X_train), columns=imputer.get_feature_names_out(self.X_train.columns)) 
                self.X_test = pd.DataFrame(imputer.transform(self.X_test), columns = imputer.get_feature_names_out(self.X_test.columns))
                self.X_val = pd.DataFrame(imputer.transform(self.X_val), columns = imputer.get_feature_names_out(self.X_val.columns))
            
            elif imputer == "mean":
                imputer = SimpleImputer(strategy='mean')
                self.X_train = pd.DataFrame(imputer.fit_transform(self.X_train), columns=imputer.get_feature_names_out(self.X_train.columns)) #auf Train fit_transform
                self.X_test = pd.DataFrame(imputer.transform(self.X_test), columns = imputer.get_feature_names_out(self.X_test.columns)) # auf test nur transform 
                self.X_val = pd.DataFrame(imputer.transform(self.X_val), columns = imputer.get_feature_names_out(self.X_val.columns)) # auf val nur transform 
    
            elif imputer == "KNN":
                print("Attention, for KNN: X_train and X_test will be sacled with StandardScaler, but I inverse transform the scale back")
                scaler = StandardScaler() # or MinMaxScaler()?
                self.X_train = pd.DataFrame(scaler.fit_transform(self.X_train), columns = self.X_train.columns)
                self.X_test = pd.DataFrame(scaler.transform(self.X_test), columns = self.X_test.columns)
                self.X_val = pd.DataFrame(scaler.transform(self.X_val), columns = self.X_val.columns)

                imputer = KNNImputer(n_neighbors = 5, weights = "distance")
                self.X_train = pd.DataFrame(imputer.fit_transform(self.X_train), columns=imputer.get_feature_names_out(self.X_train.columns))
                self.X_test = pd.DataFrame(imputer.transform(self.X_test), columns=imputer.get_feature_names_out(self.X_test.columns))
                self.X_val = pd.DataFrame(imputer.transform(self.X_val), columns=imputer.get_feature_names_out(self.X_val.columns))

                # If needed, inverse transform to get the data back to its original scale
                self.X_train = pd.DataFrame(scaler.inverse_transform(self.X_train), columns=imputer.get_feature_names_out(self.X_train.columns))
                self.X_test = pd.DataFrame(scaler.inverse_transform(self.X_test), columns=imputer.get_feature_names_out(self.X_test.columns))
                self.X_val = pd.DataFrame(scaler.inverse_transform(self.X_val), columns=imputer.get_feature_names_out(self.X_val.columns))
            
            elif imputer == "iterative":
                # Initialize the imputer
                imputer = IterativeImputer(max_iter=10, random_state=42)
                # Fit on the training data and transform both train and test data
                self.X_train = pd.DataFrame(imputer.fit_transform(self.X_train), columns=imputer.get_feature_names_out(self.X_train.columns)) #auf Train fit_transform
                self.X_test = pd.DataFrame(imputer.transform(self.X_test), columns = imputer.get_feature_names_out(self.X_test.columns)) # auf test nur transform 
                self.X_val = pd.DataFrame(imputer.transform(self.X_val), columns = imputer.get_feature_names_out(self.X_val.columns)) # auf val nur transform 

            else:
                raise Exception(f'Invalid imputer type "{imputer}"')
         
        #Target as category
        self.y_train = self.y_train.astype("category")
        self.y_test = self.y_test.astype("category")
        self.y_val = self.y_val.astype("category")
        
        #Rest the index!!!
        self.X_train = self.X_train.reset_index(drop=True)
        self.y_train = self.y_train.reset_index(drop=True)
        self.X_test = self.X_test.reset_index(drop= True)
        self.y_test = self.y_test.reset_index(drop=True)
        self.X_val = self.X_val.reset_index(drop= True)
        self.y_val = self.y_val.reset_index(drop=True)
        print("Index Adjusted!")

        # Add the indicator variables back to the dataframe
        if imputer != "drop_all":
            for col in df_NA_train.columns.values:
                if str(col).endswith("_NA"):
                    self.X_train[col] = df_NA_train[col].reset_index(drop=True)
                    self.X_test[col] = df_NA_test[col].reset_index(drop=True)
                    self.X_val[col] = df_NA_val[col].reset_index(drop=True)
        
        print("========================================")
        print("Missing Values Imputed, new Shape is:")
        print(f"Shape of X_train is {self.X_train.shape}, and y_train is: {len(self.y_train)}")
        print(f"Shape of X_val is: {self.X_val.shape} and y_val is: {len(self.y_val)}")
        print(f"Shape of X_test is:{self.X_test.shape} and y_test is: {len(self.y_test)}")
        print("========================================")
        

    def convert_to_categorical(self, list_features):
        #categorical_features = self.X.select_dtypes(include='int64').columns.tolist()
        for feature in list_features:
            self.X[feature] = self.X[feature].astype('category')

    def _get_group(self, year):
        return self.data.groupby(self.data.Date.dt.year).get_group(year)
    
    def get_year_dataset(self, year):
        year_df = OurDataset()
        year_df.set_data(self._get_group(year))
        return year_df
    
    def drop_year_dataset(self, year):
        new_ds = OurDataset()
        new_ds.set_data(self.data[self.data["Date"].dt.year != year])
        return new_ds
        
    def list_years(self):
        return list(self.data['Date'].dt.year.unique())
    
    def remove_columns(self, columns):
        self.set_data(self.data.drop(columns=columns, axis = 1))
        
    def clean_multiproducts(self):
        idx_to_remove = ~self.data['Bloomberg Ticker'].str.contains("XXXXXX", case=False, na=False) # anonymized
        idx_to_remove |= self.data['Company Name'].str.contains("XXXXXX", case=False, na=False) # anonymized
        idx_to_remove |= self.data['Company Name'].str.contains("XXXXXX", case=False, na=False) # anonymized
        idx_to_remove |= self.data['Company Name'].str.contains("XXXXXX", case=False, na=False) # anonymized
        idx_to_remove |= self.data['Company Name'].str.contains("XXXXXX", case=False, na=False) # anonymized

        multiproduct_df = self.data.loc[idx_to_remove]
        multiproduct_df.to_excel("multiproduct.xlsx")
        self.set_data(self.data[~idx_to_remove])
        print("========================================")
        print("Multiproducts are removed, only Stocks are left, new shape is: ", self.data.shape)
        print("========================================")

    def remove_stocks_with_lacking_data(self, min_datapoints=2):
        indicators = self.X.columns
        idx_to_remove = self.data[indicators].isna().sum(axis=1).ge(len(indicators.to_list()) - min_datapoints + 1)
        single_indicator_df = self.data[idx_to_remove]
        single_indicator_df.to_excel("single_indicator.xlsx")
        self.set_data(self.data[~idx_to_remove])
        print("========================================")
        print(f"Stocks with less than {min_datapoints} indicators", len(single_indicator_df))
        print(f"Stock with less than {min_datapoints} are removed, new shape is:", self.data.shape)
        print("========================================")

    def plot_missing_data(self, X_train, X_val, X_test):
        # Calculate total data points and missing data points for each set
        total_train = X_train.size
        missing_train = X_train.isnull().sum().sum()
        na_percentage_total_train = (X_train.isna().sum().sum() / X_train.size) * 100
        print("Missing Data for X_train:", na_percentage_total_train)

        total_val = X_val.size
        missing_val = X_val.isnull().sum().sum()
        na_percentage_total_val = (X_val.isna().sum().sum() / X_val.size) * 100
        print("Missing Data for X_val:", na_percentage_total_val)
        
        total_test = X_test.size
        missing_test = X_test.isnull().sum().sum()
        na_percentage_total_test = (X_test.isna().sum().sum() / X_test.size) * 100
        print("Missing Data for X_test:", na_percentage_total_test)
        
        # Data for plotting
        datasets = ['Training', 'Validation', 'Testing']
        total_data = [total_train, total_val, total_test]
        missing_data = [missing_train, missing_val, missing_test]
        
        barWidth = 0.3
        
        # Set position of bar on X axis
        r1 = np.arange(len(total_data))
        r2 = [x + barWidth for x in r1]
        
        # Make the plot
        plt.bar(r1, total_data, color='blue', width=barWidth, edgecolor='grey', label='Total Data Points')
        plt.bar(r2, missing_data, color='red', width=barWidth, edgecolor='grey', label='Missing Data Points')
        
        # Adding labels
        plt.xlabel('Datasets', fontweight='bold', fontsize=15)
        plt.ylabel('Count', fontweight='bold', fontsize=15)
        plt.xticks([r + barWidth/2 for r in range(len(total_data))], datasets)
        
        # Create legend & Show graphic
        plt.legend()
        plt.show()
    
    def plot_class_distribution(self, y_train, y_val, y_test):
        # Calculate class distribution for each set
        class_0_train = (y_train == 0).sum()
        class_1_train = (y_train == 1).sum()
        
        class_0_val = (y_val == 0).sum()
        class_1_val = (y_val == 1).sum()
        
        class_0_test = (y_test == 0).sum()
        class_1_test = (y_test == 1).sum()

        print("y_train:" , y_train.value_counts())
        class_counts_normalized_train = y_train.value_counts(normalize=True)
        print("y_train", class_counts_normalized_train)
        print("y_val:", y_val.value_counts())
        class_counts_normalized_val = y_val.value_counts(normalize=True)
        print("y_val: ", class_counts_normalized_val)
        print("y_test:" , y_test.value_counts())
        class_counts_normalized_test = y_test.value_counts(normalize=True)
        print("y_test: ", class_counts_normalized_test)
        
        # Data for plotting
        datasets = ['Training', 'Validation', 'Testing']
        class_0 = [class_0_train, class_0_val, class_0_test]
        class_1 = [class_1_train, class_1_val, class_1_test]
        
        barWidth = 0.3
        
        # Set position of bar on X axis
        r1 = np.arange(len(class_0))
        r2 = [x + barWidth for x in r1]
        
        # Make the plot
        plt.bar(r1, class_0, color='blue', width=barWidth, edgecolor='grey', label='Class Sell=0')
        plt.bar(r2, class_1, color='orange', width=barWidth, edgecolor='grey', label='Class Buy=1')
        
        # Adding labels
        plt.xlabel('Datasets', fontweight='bold', fontsize=15)
        plt.ylabel('Count', fontweight='bold', fontsize=15)
        plt.xticks([r + barWidth/2 for r in range(len(class_0))], datasets)
        
        # Create legend & Show graphic
        plt.legend()
        plt.show()

    def print_start_end_time(self, X_train, X_val, X_test):
            same_index_train = self.data.merge(X_train, how="inner", left_index=True, right_index=True)
            same_index_val = self.data.merge(X_val, how="inner", left_index=True, right_index=True)
            same_index_test = self.data.merge(X_test, how="inner", left_index=True, right_index=True)
            print(f"X_train: First date: {same_index_train['Date'].min()}, Last date: {same_index_train['Date'].max()}, X_train: {X_train.shape[0]}")
            print(f"X_val: First date: {same_index_val['Date'].min()}, Last date: {same_index_val['Date'].max()}, X_val: {X_val.shape[0]}")
            print(f"X_test: First date: {same_index_test['Date'].min()}, Last date: {same_index_test['Date'].max()}, X_test: {X_test.shape[0]}")
        
        

# Exploratory Data Analysis (EDA)

## Some Time-Series Tests and Plots

In [ ]:
#Seasonal Decomposition**:
### Could not implement seasonal_decomposition since Date is has duplicates since only day/month/year are given and no minutes/hour/seconds

## Time-Series-Plot:
def plot_time_series(X):
    num_plots = X.shape[1]
    cols = int(np.ceil(np.sqrt(num_plots)))
    rows = int(np.ceil(num_plots / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(15, 15))

    for idx, col in enumerate(X.columns):
        i, j = divmod(idx, cols)
        X[col].plot(ax=axes[i, j], title=f'Time Series for {col}')
        axes[i, j].set_ylabel(col)
        axes[i, j].set_xlabel('Time')
        axes[i, j].grid(True)
    
    # Remove extra subplots
    for idx in range(num_plots, rows * cols):
        i, j = divmod(idx, cols)
        fig.delaxes(axes[i, j])

    plt.tight_layout()
    plt.show()

#Autocorrelation and Partial Autocorrelation Plots
def plot_autocorrelation(X):
    rows = X.shape[1]
    fig, axes = plt.subplots(rows, 2, figsize=(15, 60))
    
    for idx, col in enumerate(X.columns):
        plot_acf(X[col], ax=axes[idx, 0])
        plot_pacf(X[col], ax=axes[idx, 1])
        
        axes[idx, 0].set_title(f'ACF for {col}')
        axes[idx, 1].set_title(f'PACF for {col}')

    plt.tight_layout()
    plt.show()

# Dickey-Fuller Test. This is a statistical test to check the stationarity of a time series.
def test_stationarity(X_clean):
    for col in X_clean.columns:
        print(f"Results of Dickey-Fuller Test for {col}:")
        dftest = adfuller(X_clean[col], autolag='AIC')
        dfoutput = pd.Series(dftest[0:4], index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
        for key, value in dftest[4].items():
            dfoutput[f'Critical Value ({key})'] = value
        print(dfoutput.apply(lambda x: "{0:.6f}".format(x) if abs(x) > 0.000001 else "{:e}".format(x)))
        print('---------------------------------------------------------------')


## Missing Data Plots

In [ ]:
def plot_missing_data_heatmap(df):
    
    X = df.drop(['Date', 'Company Name', 'Bloomberg Ticker',
                 'ISIN', 'CIQ ISIN', 'Acquired', 'Target'], axis=1)

    # Set Seaborn styling
    sns.set_style("whitegrid")
    plt.style.use('seaborn-notebook')

    # Heatmap visualization
    plt.figure(figsize=(25, 10))
    # Extract unique years from the 'Date' column
    unique_years = df['Date'].dt.year.unique()
    # Generate ticks positions corresponding to the start of each unique year
    ticks_positions = [df[df['Date'].dt.year == year].index[0] for year in unique_years]
    # Create the heatmap
    sns.heatmap(X.isnull().transpose(), cmap='YlGnBu', cbar_kws={'label': 'Missing Values'}, xticklabels=False)
    # Set x-axis labels and ticks
    plt.xticks(ticks_positions, unique_years)
    plt.xlabel('Years')
    plt.title("Heatmap of Missing Values")
    plt.show()

def plot_missing_data_tabel(X):

    print(f'There are in total {X.isnull().sum().sum()} NAN in the indicator dataframe')
    # Percentage calculations
    na_percent = X.isna().sum() / X.shape[0] * 100
    zero_percent = X.isin([0]).sum() / X.shape[0] * 100
    zero_and_na_percent = (X.isna().sum() + X.isin([0]).sum()) / X.shape[0] * 100

    # Create missing data summary
    missing_data = pd.DataFrame({
        'total_NA': X.isna().sum(),
        '%_NA': na_percent,
        'total_0': X.isin([0]).sum(),
        '%_0': zero_percent,
        'Total_NA_and_0': (X.isna().sum() + X.isin([0]).sum()),
        '%_NA_and_0': zero_and_na_percent
    }).sort_values(by='%_NA_and_0', ascending=False)

    print(missing_data)

    # Bar chart visualization
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(30, 10))

    na_labels = na_percent.sort_values(ascending=False).index.values.tolist()
    ax[0].bar(na_labels, na_percent.sort_values(ascending=False).values, color='skyblue')
    ax[0].set_ylabel("NA Percentage")
    ax[0].set_title("Features with the highest NA Percentage")
    ax[0].tick_params(axis='x', rotation=90)

    ax[1].bar(na_labels, zero_percent.sort_values(ascending=False).values, color='salmon')
    ax[1].set_ylabel("0 Value Percentage")
    ax[1].set_title("Features with the highest 0 Value Percentage")
    ax[1].tick_params(axis='x', rotation=90)

    ax[2].bar(na_labels, zero_and_na_percent.sort_values(ascending=False).values, color='lightgreen')
    ax[2].set_ylabel("NA & 0 Value Percentage")
    ax[2].set_title("Features with the highest NA & 0 Value Percentage")
    ax[2].tick_params(axis='x', rotation=90)

    plt.tight_layout()
    plt.show()

def plot_missing_data_simple(X):
    # Set Seaborn styling
    sns.set_style("whitegrid")
    plt.style.use('seaborn-notebook')

    # Percentage calculations
    na_percent = X.isna().sum() / X.shape[0] * 100

    # Bar chart visualization
    fig, ax = plt.subplots(figsize=(20, 10))
    
    na_labels = na_percent.sort_values(ascending=False).index.values.tolist()
    bars = ax.bar(na_labels, na_percent.sort_values(ascending=False).values, color=sns.color_palette("coolwarm_r", len(na_labels)))
    
    # Adding the exact percentage on top of each bar
    for bar in bars:
        height = bar.get_height()
        ax.annotate(f'{height:.2f}%',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom',
                    fontsize=10)
    
    ax.set_ylabel("NA Percentage", fontsize=14)
    ax.set_title("Features with the Highest NA Percentage", fontsize=16)
    ax.tick_params(axis='x', rotation=90, labelsize=12)
    ax.tick_params(axis='y', labelsize=12)
    
    plt.tight_layout()
    plt.show()



### Binary PCA Plot of Missing Data

In [ ]:
def binary_missing_matrix(X):
    return X.isna().astype(int)

def perform_pca_on_missing_data(binary_matrix, n_components=2):
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(binary_matrix)
    return principal_components, pca  # Return the PCA object as well

def plot_pca(principal_components, pca, X, y):
    plt.figure(figsize=(12, 8))
    
    # Create a color map based on 'Target' values
    colors = ['blue' if val == 0 else 'orange' for val in y]
    
    plt.scatter(principal_components[:, 0], principal_components[:, 1], alpha=0.5, c=colors)
    
    # Plot the arrows (loadings)
    for i, (comp1, comp2) in enumerate(zip(pca.components_[0], pca.components_[1])):
        plt.arrow(0, 0, comp1, comp2, head_width=0.02, head_length=0.03, color='red')
        if abs(comp1) > 0.2 or abs(comp2) > 0.2:  # only label arrows that are long enough
            plt.text(comp1*1.2, comp2*1.2, X.columns[i], color='black', ha='center', va='center')
    
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.title('PCA of Binary Missing Data Representation\nExplained Variance: {:.2%} and {:.2%}'.format(pca.explained_variance_ratio_[0], pca.explained_variance_ratio_[1]))
    
    # Create a legend for the plot
    plt.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Sell (0)'),
                        plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='orange', markersize=10, label='Buy (1)')],
               loc='upper right')
    
    plt.grid(True)
    plt.show()

def plot_missing_data_pca(X, y):
    # 1. Create a binary matrix of missing values
    binary_matrix = binary_missing_matrix(X)
    
    # 2. Perform PCA on the binary matrix
    principal_components, pca = perform_pca_on_missing_data(binary_matrix)
    
    # 3. Plot the results
    plot_pca(principal_components, pca, X, y)

In [ ]:
def binary_missing_matrix(X):
    return X.isna().astype(int)

def perform_pca_on_missing_data(binary_matrix, n_components=3):  # Default changed to 3 components
    pca = PCA(n_components=n_components)
    principal_components = pca.fit_transform(binary_matrix)
    return principal_components, pca

def plot_pca_3d_only_arrows(principal_components, pca, X):
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot the arrows (loadings) only
    max_length = 0  # This will help set axis limits later
    for i, (comp1, comp2, comp3) in enumerate(zip(pca.components_[0], pca.components_[1], pca.components_[2])):
        arrow_length = np.sqrt(comp1**2 + comp2**2 + comp3**2)
        max_length = max(max_length, arrow_length)
        ax.quiver(0, 0, 0, comp1, comp2, comp3, length=arrow_length, arrow_length_ratio=0.1, color='red', linewidth=1.5)
        ax.text(comp1*1.2, comp2*1.2, comp3*1.2, X.columns[i], color='black', ha='center', va='center')

    # Set axis limits based on the maximum arrow length
    ax.set_xlim([-max_length, max_length])
    ax.set_ylim([-max_length, max_length])
    ax.set_zlim([-max_length, max_length])

    ax.set_xlabel('Principal Component 1')
    ax.set_ylabel('Principal Component 2')
    ax.set_zlabel('Principal Component 3')
    ax.set_title('Loadings in PCA Space for Binary Missing Data Representation')

    plt.show()

def plot_missing_data_pca_3d_only_arrows(X, y):  # Name changed to signify 3D plotting
    binary_matrix = binary_missing_matrix(X)
    principal_components, pca = perform_pca_on_missing_data(binary_matrix)
    plot_pca_3d_only_arrows(principal_components, pca, X)  # Call 3D plot function


def plot_pca_3d(principal_components, pca, X, y):
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    colors = ['blue' if val == 0 else 'orange' for val in y]
    ax.scatter(principal_components[:, 0], principal_components[:, 1], principal_components[:, 2], alpha=0.5, c=colors)
    
    ax.set_xlabel('Principal Component 1')
    ax.set_ylabel('Principal Component 2')
    ax.set_zlabel('Principal Component 3')
    
    title = 'PCA of Binary Missing Data Representation\nExplained Variance: {:.2%}, {:.2%}, and {:.2%}'.format(
        pca.explained_variance_ratio_[0], pca.explained_variance_ratio_[1], pca.explained_variance_ratio_[2])
    ax.set_title(title)
    
    # Handling the legend
    ax.legend(handles=[plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=10, label='Sell (0)'),
                       plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='orange', markersize=10, label='Buy (1)')],
               loc='upper right')

    plt.show()

def plot_missing_data_pca_3d(X, y):  # Name changed to signify 3D plotting
    binary_matrix = binary_missing_matrix(X)
    principal_components, pca = perform_pca_on_missing_data(binary_matrix)
    plot_pca_3d(principal_components, pca, X, y)  # Call 3D plot function

### Class Imbalnce


In [ ]:
def plot_target_distribution_yearly(df):
    X = df.drop( ['Date',  'Company Name', 'Bloomberg Ticker',
                             'ISIN', 'CIQ ISIN', 'Acquired', 'Target'], axis = 1)
    # Group by year and get unique years
    dfgroups = df.groupby(df['Date'].dt.year)
    years = df['Date'].dt.year.unique()

   # Determine the plotting grid dynamically
    num_plots = X.shape[1]
    cols = int(np.ceil(np.sqrt(num_plots)))
    rows = int(np.ceil(num_plots / cols))
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 15))
    axes = axes.ravel()

    for index, year in enumerate(years):
        year_df = dfgroups.get_group(year)
        sns.barplot(x=year_df['Target'].value_counts().index, 
                    y=year_df['Target'].value_counts(normalize=True).values, 
                    ax=axes[index])
        axes[index].set_title(year)
        axes[index].set_xlabel("Target")
        axes[index].set_ylabel("Proportion")
        axes[index].set_xticklabels(["Sell", "Buy"])
        axes[index].set_ylim(0, 1)

    # Handling any extra subplots
    for i in range(len(years), rows * cols):
        fig.delaxes(axes[i])

    plt.tight_layout()
    plt.suptitle("Buy/Sell Proportions over the Years", y=1.05)
    plt.show()

def plot_traget_distribution_simple(y):
    # Display counts of target values
    print(y.value_counts())
    class_counts_normalized = y.value_counts(normalize=True)
    print(class_counts_normalized)
    # Display skewness and kurtosis
    print("Skewness: %f" % y.astype(int).skew()) #does not make sense for binary classification
    print("Kurtosis: %f" % y.astype(int).kurt()) #does not make sense for binary classification
    # Distribution of target
    plt.figure(figsize=(8, 5))
    sns.countplot(x=y)
    plt.title('Distribution of Target')
    plt.xticks(ticks=[0,1], labels=['Sell', 'Buy'])
    plt.show()

   

### Histograms and Distribution for each Feature

In [ ]:
def plot_mean_median_hist(X, y):
    num_plots = X.shape[1]
    cols = int(np.ceil(np.sqrt(num_plots)))
    rows = int(np.ceil(num_plots / cols))
    
    fig, ax = plt.subplots(rows, cols, figsize=(20, 15))
    fig.subplots_adjust(hspace=0.4, wspace=0.4)

    for i, col in enumerate(X.columns):
        j, k = divmod(i, cols)

        feat_cl0 = X.loc[y == 0, col]
        feat_cl1 = X.loc[y == 1, col]

        mean_cl0 = feat_cl0.mean()
        mean_cl1 = feat_cl1.mean()

        med_cl0 = feat_cl0.median()
        med_cl1 = feat_cl1.median() 

        plot_range = (min(feat_cl0.quantile(0.05), feat_cl1.quantile(0.05)),
                      max(feat_cl0.quantile(0.95), feat_cl1.quantile(0.95)))

        ax[j, k].hist(feat_cl0, color="royalblue", bins=50, alpha=0.6, range=plot_range, label="Sell")
        ax[j, k].hist(feat_cl1, color="darkorange", bins=50, alpha=0.6, range=plot_range, label="Buy")
        ax[j, k].axvline(x=mean_cl0, color="blue", linestyle=':', linewidth=2, label=f"mean {round(mean_cl0, 3)}")
        ax[j, k].axvline(x=med_cl0, color="blue", linestyle='--', linewidth=2, label=f"median {round(med_cl0, 3)}")
        ax[j, k].axvline(x=mean_cl1, color="orange", linestyle=':', linewidth=2, label=f"mean {round(mean_cl1, 3)}")
        ax[j, k].axvline(x=med_cl1, color="orange", linestyle='--', linewidth=2, label=f"median {round(med_cl1, 3)}")
        
        ax[j, k].set_xlabel(col, fontsize=12)
        ax[j, k].tick_params(axis='x', rotation=45)
        ax[j, k].grid(axis='y', linestyle='--', alpha=0.7)
        ax[j, k].legend(fontsize=10, loc='upper right')

    # Remove extra subplots
    for l in range(i+1, rows*cols):
        j, k = divmod(l, cols)
        ax[j, k].axis('off')

    plt.show()

def plot_histogram_scaled_for_KDE(X):
    #With this modification, the KDE is plotted on a different y-scale, which should make it more visible and easier to interpret alongside the histogram.
    num_plots = X.shape[1]
    cols = int(np.ceil(np.sqrt(num_plots)))
    rows = int(np.ceil(num_plots / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(15, 15))

    for idx, col in enumerate(X.columns):
        i, j = divmod(idx, cols)
        
        # Histogram without KDE
        sns.histplot(X[col], color="skyblue", ax=axes[i, j], label="Histogram")
        
        # Create a secondary y-axis for the KDE
        ax2 = axes[i, j].twinx()
        sns.kdeplot(X[col], ax=ax2, color="red", label="KDE")
        
        # Titles and labels
        axes[i, j].set_title(f'Histogram for {col}')
        axes[i, j].set_xlabel(col)
        axes[i, j].set_ylabel('Count')
        ax2.set_ylabel('Density')
        axes[i, j].grid(True)

    # Remove extra subplots
    for idx in range(num_plots, rows * cols):
        i, j = divmod(idx, cols)
        fig.delaxes(axes[i, j])

    plt.tight_layout()
    plt.show()

def plot_histogram(X):
    num_plots = X.shape[1]
    cols = int(np.ceil(np.sqrt(num_plots)))
    rows = int(np.ceil(num_plots / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(15, 15))

    for idx, col in enumerate(X.columns):
        i, j = divmod(idx, cols)
        
        # Histogram with normalization and transparency
        sns.histplot(X[col], color="skyblue", ax=axes[i, j], label="Histogram", ) #stat="probability", kde=False
        
        # Titles and labels
        axes[i, j].set_title(f'Histogram for {col}')
        axes[i, j].set_xlabel(col)
        axes[i, j].set_ylabel('Count') #Density if stats=probablility
        axes[i, j].grid(True)
        #axes[i, j].legend()

    # Remove extra subplots
    for idx in range(num_plots, rows * cols):
        i, j = divmod(idx, cols)
        fig.delaxes(axes[i, j])

    plt.tight_layout()
    plt.show()

def plot_hist_yearly(df):
    # Filtering out unnecessary columns
    X = df.drop(['Date', 'Company Name', 'Bloomberg Ticker', 'ISIN', 'CIQ ISIN', 'Acquired', 'Target'], axis=1)
    
    dfgroups = df.groupby(df['Date'].dt.year)
    years = sorted(df['Date'].dt.year.unique())
    topics = list(X.select_dtypes(include=[float]))

    for topic in topics:
        fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(30, 20))
        fig.suptitle(f'{topic} with Kernel Density and Median (red)', fontsize='large')
        ax = ax.ravel()  # flatten the axis array

        # Plot each year's histogram
        for index, year in enumerate(years):
            year_df = dfgroups.get_group(year)
            ax[index].set_title(year)
            sns.histplot(year_df[topic], kde=True, ax=ax[index])
            ax[index].axvline(x=year_df[topic].median(), color="red", label="Median", linestyle='-')
            
            # Plot density of all years together to compare them
            sns.kdeplot(year_df[topic], label=year, ax=ax[-1], legend=False)
        
        # Set properties for the last subplot
        ax[-1].legend(years, fontsize='medium')
        ax[-1].set_title("Density Comparison", fontsize='medium')
        
        # Remove unused subplots
        for unused_ax in ax[len(years):-1]:  # iterate over unused subplots (excluding the comparison plot)
            fig.delaxes(unused_ax)

        plt.tight_layout()
        plt.show()

### Boxplots and Rolling Statisitcs

In [ ]:
def rolling_stats(df, window=30):

    X = df.drop(['Date', 'Company Name', 'Bloomberg Ticker',
                 'ISIN', 'CIQ ISIN', 'Acquired', 'Target'], axis=1)
    
    # Set Date from df as the index for X
    X = X.set_index(df['Date']) 
    num_plots = X.shape[1]
    cols = int(np.ceil(np.sqrt(num_plots)))
    rows = int(np.ceil(num_plots / cols))
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 15))

    for idx, col in enumerate(X.columns):
        i, j = divmod(idx, cols)
        
        X[col].plot(ax=axes[i, j], label='Original')
        X[col].rolling(window=window).mean().plot(ax=axes[i, j], label='Rolling Mean')
        X[col].rolling(window=window).std().plot(ax=axes[i, j], label='Rolling Std')
        axes[i, j].set_title(f'Rolling Stats for {col}')
        axes[i, j].legend()
        axes[i, j].grid(True)

    # Remove extra subplots
    for idx in range(num_plots, rows * cols):
        i, j = divmod(idx, cols)
        fig.delaxes(axes[i, j])

    plt.tight_layout()
    plt.show()


def plot_boxplot(X):
    num_plots = X.shape[1]
    cols = int(np.ceil(np.sqrt(num_plots)))
    rows = int(np.ceil(num_plots / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(15, 15))

    for idx, col in enumerate(X.columns):
        i, j = divmod(idx, cols)
        sns.boxplot(data=X[col], ax=axes[i, j])  # Here's the change. Specify the column for data.
        axes[i, j].set_title(f' Boxplot for {col}')
        axes[i, j].grid(True)

    # Remove extra subplots
    for idx in range(num_plots, rows * cols):
        i, j = divmod(idx, cols)
        fig.delaxes(axes[i, j])

    plt.tight_layout()
    plt.show()

# Assuming 'Date' column exists in `df` and is of `datetime64` type
def box_plot_yearly(df):
    df_new = df.copy()  # Create a copy to prevent modifying the original dataframe
    df_new['year'] = df_new['Date'].dt.year
    
    indicators = df_new.drop(['Date', 'year', 'Company Name', 'Bloomberg Ticker',
                              'ISIN', 'CIQ ISIN', 'Acquired', 'Target'], axis=1)

    num_plots = indicators.shape[1]
    cols = int(np.ceil(np.sqrt(num_plots)))
    rows = int(np.ceil(num_plots / cols))
    fig, axes = plt.subplots(rows, cols, figsize=(15, 15))

    for idx, col in enumerate(indicators.columns):
        i, j = divmod(idx, cols)
        sns.boxplot(x='year', y=col, data=df_new, ax=axes[i, j])
        
        # Shorten the x-tick labels
        # Get unique years and shorten them
        unique_years = df_new['year'].unique()
        shortened_years = [str(year)[-2:] for year in unique_years]
        axes[i, j].set_xticks(np.arange(len(unique_years)))  # Set ticks positions
        axes[i, j].set_xticklabels(shortened_years)  # Set shortened year labels
        
        axes[i, j].set_title(f'Yearly Boxplot for {col}')
        axes[i, j].grid(True)

    # Remove extra subplots
    for idx in range(num_plots, rows * cols):
        i, j = divmod(idx, cols)
        fig.delaxes(axes[i, j])

    plt.tight_layout()
    plt.show()

### Quick Outlier Removal

In [ ]:
def remove_outliers_IQR(df, threshold):
    # Store columns with outliers for later printing
    outliers_dict = {}
    
    for column in df.columns:
        # Exclude NaN values for the current column
        column_data = df[column].dropna()
        
        # Calculate Q1 and Q3
        Q1 = column_data.quantile(0.25)
        Q3 = column_data.quantile(0.75)
        IQR = Q3 - Q1
        
        # Define bounds for the outliers
        lower_bound = Q1 - threshold * IQR ## 1.5 or 3.0
        upper_bound = Q3 + threshold * IQR
        
        # Find outliers (excluding NaN values)
        outliers = df.loc[column_data.index][(column_data < lower_bound) | (column_data > upper_bound)]
        
        # If there are outliers, print and store them
        if not outliers.empty:
            #print(f"Outliers for {column}:")
            #print(outliers)
            outliers_dict[column] = outliers
        
        # Remove the outliers from the dataframe
        df = df.drop(outliers.index)
    
    if not outliers_dict:
        print("No outliers found.")
    return df

def remove_outliers_with_zscore(df, threshold=3):
    # Store columns with outliers for later printing
    outliers_dict = {}
    
    for column in df.columns:
        # Skip columns with non-numeric data types
        if pd.api.types.is_numeric_dtype(df[column]):
            # Calculate z-scores
            zs = zscore(df[column].dropna())
            
            # Identify the outliers
            outliers = df[column].dropna()[(abs(zs) > threshold)]
            
            # If there are outliers, print and store them
            if not outliers.empty:
                print(f"Outliers for {column}:")
                print(outliers)
                outliers_dict[column] = outliers
                
                # Remove the rows with outliers
                df = df.drop(outliers.index)
    
    if not outliers_dict:
        print("No outliers found.")
        
    return df


### Correlation Heatmap

In [ ]:
def plot_correlation(X):
    plt.figure(figsize=(15, 10))
    corr = pd.DataFrame(X).corr()
    heatmap = sns.heatmap(corr, vmin=-1, vmax=1, center=0, annot=True, cmap="RdBu_r")
    heatmap.set_title("Correlation Heatmap", fontdict={"fontsize":16})
    plt.show()

def plot_correlation_triangle(X):
    # Modify the mask to hide only the strict upper triangle
    mask = np.triu(np.ones_like(X.corr(), dtype=bool), k=1)
    
    plt.figure(figsize=(16, 6))
    heatmap = sns.heatmap(X.corr(), mask=mask, vmin=-1, vmax=1, annot=True, center=0, cmap='RdBu_r')
    heatmap.set_title('Triangle Correlation Heatmap')
    plt.show()


### Feature Importance with Univariate Statistical Tests
This method involves evaluating the statistical relationship between each feature and the target variable independently. Common statistical tests such as chi-square, ANOVA, or correlation coefficients can be used to quantify the strength of the relationship. Features with higher test statistics or p-values below a certain threshold are considered more important. For instance, here we use chi-square test for categorical features and ANOVA for numerical features.

https://medium.com/datadriveninvestor/five-approaches-to-determine-feature-importance-in-classification-problems-7be5daef37a6

In [ ]:
from scipy.stats import kruskal
from scipy.stats import f_oneway, yeojohnson
from sklearn.preprocessing import FunctionTransformer

def categorical_feature_importance(X, y):
    categorical_features = X.select_dtypes(include=['category', 'object']).columns

    chi2_scores, p_values = [], []
    
    for feature in categorical_features:
        contingency_table = pd.crosstab(X[feature], y)
        chi2, p_value, _, _ = chi2_contingency(contingency_table)
        chi2_scores.append(chi2)
        p_values.append(p_value)

    results = pd.DataFrame({'Feature': categorical_features, 'Chi2 Score': chi2_scores, 'P-value': p_values})
    cat_res = results.sort_values(by='P-value')

    print("Categorical Features:")
    print(cat_res)

def one_way_anova_with_yeo_johnson_trafo(X, y):
    numerical_features = X.select_dtypes(include=[float, int]).columns
    f_scores, p_values = [], []

    for feature in numerical_features:
        groups = [X[y == c][feature].dropna() for c in np.unique(y)]
        transformed_groups = [yeojohnson(group)[0] for group in groups]
        f, p_value = f_oneway(*transformed_groups)
        f_scores.append(f)
        p_values.append(p_value)

    results = pd.DataFrame({'Feature': numerical_features, 'F-score': f_scores, 'P-value': p_values})
    num_res = results.sort_values(by='P-value')

    print("\nNumerical Features:")
    print(num_res)

def numerical_feature_importance_kruskal(X, y):
    numerical_features = X.select_dtypes(include=[float, int]).columns
    h_stats, p_values = [], []

    for feature in numerical_features:
        groups = [X[y == c][feature].dropna() for c in np.unique(y)]
        H, p_value = kruskal(*groups)
        h_stats.append(H)
        p_values.append(p_value)

    results = pd.DataFrame({'Feature': numerical_features, 'H-statistic': h_stats, 'P-value': p_values})
    num_res = results.sort_values(by='P-value')

    print("\nNumerical Features:")
    print(num_res)


def feature_importance_stats_test(X_train, y_train):
    #categorical_features = X_train.select_dtypes(include='category').columns
    #numerical_features = X_train.select_dtypes(include=float).columns
    categorical_feature_importance(X_train, y_train)
    print("---------------------------------------------")
    one_way_anova_with_yeo_johnson_trafo(X_train, y_train)
    print("---------------------------------------------")
    numerical_feature_importance_kruskal(X_train, y_train)
    


# Data Initialization

Loading of Data and Initalization of the dataset.
Preform EDA, show plots and distributions.


In [ ]:
# Initializing dataset path
path = os.path.join(sys.path[0])
# Loading and Handling dataset
mydata = OurDataset(path)

### Removing and sorting stuff
mydata.remove_timestamps()
mydata.remove_duplicates()
mydata.sort_after_date()
mydata.clean_multiproducts()
mydata.remove_stocks_with_lacking_data(min_datapoints=2)


# Drop 'Financial_Ratio_11', and 'Financial_Ratio_14' otherwise dataset is too small and has too much noise!
mydata.remove_columns(columns = ['Financial_Ratio_11', 'Financial_Ratio_14']) 
mydata.data.to_excel("Cleaned_Data.xlsx")
# Splitting the data into train, validation and test sets! Also does missing values imputation, outlier cleaning and shuffle or not shuffle!
mydata.split_dataset(imputer = "iterative", remove_outliers=True, shuffle=True)

# Renaming
X_train, X_test, X_val, y_train, y_test, y_val = mydata.X_train, mydata.X_test, mydata.X_val, mydata.y_train, mydata.y_test, mydata.y_val

###Check Format
print("----------------------------------")
print("----dTypes of X_train----------")
print(X_train.dtypes)
print("----dTypes of X_val----------")
print(X_val.dtypes)
print("----dTypes of X_test----------")
print(X_test.dtypes)
print("----------------------------------")
print("dtype y_train", y_train.dtype)
print("dtype y_val", y_val.dtype)
print("dtype y_test", y_test.dtype)
print("----------------------------------")
print("has missing values?", X_train.isnull().values.any())
print("has missing values?", X_val.isnull().values.any())
print("has missing values?", X_test.isnull().values.any())
print("----------------------------------")
print("Class Disitbution for y_train")
plot_traget_distribution_simple(y_train)
print("Class Disitbution for y_val")
plot_traget_distribution_simple(y_val)
print("Class Disitbution for y_test")
plot_traget_distribution_simple(y_test)
print("----------------------------------")
print("----------------------------------")



# Printing Plots/EDA/Validation

In [ ]:
## Plotting and Printing all the EDA Stuff
def do_eda( df, X, y):
    #MissingData Stuff
    plot_missing_data_tabel(X)
    plot_missing_data_simple(X)
    plot_missing_data_heatmap(df)
    plot_missing_data_pca(X, y)
    plot_missing_data_pca_3d(X, y)
    plot_missing_data_pca_3d_only_arrows(X, y)
    #TargetDistribution Stuff
    plot_traget_distribution_simple(y)
    plot_target_distribution_yearly(df)
    #BoxPlot Stuff
    plot_boxplot(X)
    box_plot_yearly(df)
    #Histogramm Stuff
    rolling_stats(df)
    plot_mean_median_hist(X, y) 
    plot_hist_yearly(df)
    plot_histogram_scaled_for_KDE(X)
    plot_histogram(X)
    #Correlation Matrix
    plot_correlation_triangle(X) 
  

def do_eda_validation(X_clean, y_clean):
    numerical_features = X_clean.select_dtypes(include='float64')
    plot_traget_distribution_simple(y_clean)
    #plot_histogram_scaled_for_KDE(numerical_features)
    #plot_histogram(numerical_features)
    plot_mean_median_hist(numerical_features, y_clean)
    #plot_boxplot(numerical_features)
    plot_correlation_triangle(numerical_features) 
    feature_importance_stats_test(X_clean, y_clean)

# Function to compare central tendency and spread
def compare_data_not_scaled(df1, df2):
    stats = ['mean', 'median', 'std', 'var']
    comparison_stats = pd.DataFrame(index=df1.columns, columns=pd.MultiIndex.from_product([stats, ['Not Imputed', 'Imputed']]))
    
    for col in df1.columns:
        for stat in stats:
            comparison_stats.loc[col, (stat, 'Not Imputed')] = getattr(df1[col], stat)()
            comparison_stats.loc[col, (stat, 'Imputed')] = getattr(df2[col], stat)()
    
    print(comparison_stats)
    # Plotting the comparison results
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
    fig.suptitle('Comparison of Central Tendency and Spread Before and After Imputation', y=1.03)

    for ax, stat in zip(axes.flatten(), ['mean', 'median', 'std', 'var']):
        comparison_stats[stat].plot(kind='bar', ax=ax)
        ax.set_title(stat.capitalize())
        ax.set_ylabel(stat.capitalize())
        ax.legend(title='Dataset')

    plt.tight_layout()
    plt.show()

def compare_data(df1, df2):
    stats = ['mean', 'median', 'std', 'var']
    comparison_stats = pd.DataFrame(index=df1.columns, columns=pd.MultiIndex.from_product([stats, ['Not Imputed', 'Imputed']]))
    
    for col in df1.columns:
        for stat in stats:
            if stat in ['std', 'var']:
                comparison_stats.loc[col, (stat, 'Not Imputed')] = np.log1p(getattr(df1[col], stat)())
                comparison_stats.loc[col, (stat, 'Imputed')] = np.log1p(getattr(df2[col], stat)())
            else:
                comparison_stats.loc[col, (stat, 'Not Imputed')] = getattr(df1[col], stat)()
                comparison_stats.loc[col, (stat, 'Imputed')] = getattr(df2[col], stat)()
    
    print(comparison_stats)
    # Plotting the comparison results
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 8))
    fig.suptitle('Comparison of Central Tendency and Spread Before and After Imputation', y=1.03)

    for ax, stat in zip(axes.flatten(), stats):
        if stat in ['std', 'var']:
            ax.set_yscale('log')
            ax.set_ylabel(f'Log {stat.capitalize()}')
        else:
            ax.set_ylabel(stat.capitalize())
            
        comparison_stats[stat].plot(kind='bar', ax=ax)
        ax.set_title(stat.capitalize())
        ax.legend(title='Dataset')

    plt.tight_layout()
    plt.show()

def qqplot_columns(df_not_imputed, df_imputed, cols):
    """
    Creates QQ plots and box plots for the specified columns in the non-imputed and imputed DataFrames.

    Parameters:
    df_not_imputed (pd.DataFrame): DataFrame before imputation
    df_imputed (pd.DataFrame): DataFrame after imputation
    cols (list of str): List of column names to plot
    """
    for col in cols:
        fig = plt.figure(figsize=(12, 5))
        gs = gridspec.GridSpec(1, 2, width_ratios=[3, 1]) 
        ax0 = plt.subplot(gs[0])
        
        not_imputed_sorted = np.sort(df_not_imputed[col].dropna())
        imputed_sorted = np.sort(df_imputed[col])
        
        # Compute the theoretical quantiles for not imputed data
        not_imputed_quantiles = stats.probplot(not_imputed_sorted, dist="norm")[0][0]
        # Compute the theoretical quantiles for imputed data
        imputed_quantiles = stats.probplot(imputed_sorted, dist="norm")[0][0]
        
        ax0.plot(not_imputed_quantiles, not_imputed_sorted, '.', color='blue', label='Not Imputed')
        ax0.plot(not_imputed_quantiles, np.poly1d(np.polyfit(not_imputed_quantiles, not_imputed_sorted, 1))(not_imputed_quantiles), color='blue', linestyle='--')
        
        ax0.plot(imputed_quantiles, imputed_sorted, '.', color='orange', label='Imputed')
        ax0.plot(imputed_quantiles, np.poly1d(np.polyfit(imputed_quantiles, imputed_sorted, 1))(imputed_quantiles), color='orange', linestyle='--')
        
        ax0.set_title(f'QQ Plot for {col}')
        ax0.set_xlabel('Theoretical Quantiles')
        ax0.set_ylabel(f'Sorted Values of {col}')
        ax0.legend()
        
        ax1 = plt.subplot(gs[1])
        sns.boxplot(data=pd.concat([df_not_imputed[col], df_imputed[col]], axis=1, keys=['Not Imputed', 'Imputed']), ax=ax1)
        ax1.set_title('Boxplot Comparison')
        ax1.set_ylabel(f'Values of {col}')
        
        plt.tight_layout()
        plt.show()




##############################################


#do_eda(mydata.data.copy(deep=True), mydata.X.copy(deep=True), mydata.y.copy(deep=True))
X_combined = pd.concat([X_train, X_val], axis=0, ignore_index=True)
y_combined =  pd.concat([y_train, y_val], axis =0, ignore_index=True)
X_combined.to_excel("X_combined.xlsx")
#do_eda_validation(X_combined, y_combined)

# Running the comparison
X_not_imputed_combined = pd.concat([mydata.X_train_not_imputed, mydata.X_val_not_imputed], axis=0, ignore_index=True)
#compare_data(X_not_imputed_combined, X_combined.select_dtypes(include='float64'))
# Creating QQ-plots for all columns
qqplot_columns(X_not_imputed_combined, X_combined.select_dtypes(include='float64'), X_not_imputed_combined.columns)
#####
#X_combined_all = pd.concat([X_combined, X_test], axis=0, ignore_index=True)
#test_stationarity(X_combined_all.select_dtypes(include='float64'))


# Set Parameters:
## Sampling

We use the following sampling methods:

1. RandomOverSampler: category and continuous
2. SMOTE (Synthetic Minority Over-sampling Technique) only for continuous data
3. BorderlineSMOTE only for continuous data
4. SMOTENC for categorical data!!!!


### Scaling

Scaling is a necessary pre-processing step for certain machine learning algorithms, especially those that rely on the calculation of distances or optimization methods.

Here are some examples of algorithms where scaling is crucial:
- Linear and Logistic Regression
- Support Vector Machines (SVM)
- K-Nearest Neighbors (K-NN)
- Neural Networks
- Principal Component Analysis (PCA)

However, there are also algorithms where scaling isn't necessary:
- Tree-based algorithms: Algorithms like Decision Trees, Random Forests, Gradient Boosting, and XGBoost do not require feature scaling. These algorithms are not distance-based and can handle various scales.
- Naive Bayes: Naive Bayes is not affected by the feature scales as it is not distance based.

In [ ]:
# Unified parameters for pipeline's random search
scaler = StandardScaler()
mms = MinMaxScaler()
ros = RandomOverSampler(random_state = 42) # for continuous and category 
smote = SMOTE(random_state= 42) # only for continuous data
borderline_smote = BorderlineSMOTE(random_state=42) # only for continuous data
smote_nc = SMOTENC(categorical_features="auto", random_state=42) ######## Important for Categorical Data!!!
kFold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42) # n_splits=5


# Training the Model



In [ ]:
class TrainModel:
    def __init__(self, X_train, y_train, X_test, y_test, X_val, y_val, model=None, pipeline=None, model_params=None, pipeline_params=None, verbose=0):
        # Data Attributes
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
        self.X_val = X_val
        self.y_val = y_val

        # Model and Pipeline
        self.model = model
        self.pipe = pipeline
        self.model_params = model_params
        self.pipeline_params = pipeline_params

        # Other Attributes
        self.verbose = verbose
        self.result = None  # will hold the trained model or pipeline

        # Cross-validation setup
        number_of_splits = min(10, len(X_train)) # or 5 K-Fold?
        self.kFold = StratifiedKFold(n_splits=number_of_splits, shuffle=True, random_state=42) # True or False beim Shuffle?!

    def train_with_crossvalidation(self, use_pipeline=True, use_grid_search=True): 
        # Error checking
        if use_pipeline and (not self.pipe or not self.pipeline_params):
            raise ValueError("Pipeline or pipeline parameters not provided!")
        if not use_pipeline and (not self.model or not self.model_params):
            raise ValueError("Model or model parameters not provided!")
        
        X_combined = pd.concat([self.X_train, self.X_val], axis=0, ignore_index=True).reset_index(drop=True) 
        y_combined =  pd.concat([self.y_train, self.y_val], axis=0, ignore_index=True).reset_index(drop=True) 
        

        # Select appropriate searcher
        if use_pipeline:
            estimator = self.pipe
            params = self.pipeline_params
        else:
            estimator = self.model
            params = self.model_params

        if use_grid_search:
            searcher = GridSearchCV(estimator=estimator, 
                                    param_grid=params, 
                                    scoring="f1_weighted", 
                                    cv=self.kFold, 
                                    n_jobs=-1, 
                                    verbose=self.verbose)
        else:
            searcher = RandomizedSearchCV(estimator=estimator, 
                                          param_distributions=params, 
                                          scoring="f1_weighted", 
                                          cv=self.kFold, 
                                          n_iter=30,
                                          n_jobs=-1,
                                          random_state=42, 
                                          verbose=self.verbose)
            
        searcher.fit(X_combined, y_combined)
        self.result = searcher.best_estimator_ #more efficient than just saving searcher

        # Informative print statement
        print(f"Best {'pipeline' if use_pipeline else 'model'} params used:", searcher.best_params_)

    def train_with_validation(self, use_pipeline=True, use_grid_search=True):
        """
        Train the model using a separate validation set for model selection.
        """
        # Error checking
        if use_pipeline and (not self.pipe or not self.pipeline_params):
            raise ValueError("Pipeline or pipeline parameters not provided!")
        if not use_pipeline and (not self.model or not self.model_params):
            raise ValueError("Model or model parameters not provided!")
        
        # Select appropriate model/estimator
        if use_pipeline:
            estimator = clone(self.pipe)
            params = self.pipeline_params
        else:
            estimator = clone(self.model)
            params = self.model_params
        
        # 1. Train the Model on the Training Set
        #estimator.fit(self.X_train, self.y_train) #you don't need to manually fit the model on the training set before this process as the GridSearchCV
        
        # 2. Hyperparameter Tuning on the Validation Set
        test_fold = np.hstack([-np.ones(len(self.X_train)), np.zeros(len(self.X_val))])
        ps = PredefinedSplit(test_fold)
        
        if use_grid_search:
            searcher = GridSearchCV(estimator, param_grid=params, scoring="f1_weighted", cv=ps, n_jobs=-1, verbose=self.verbose)
        else:
            searcher = RandomizedSearchCV(estimator, param_distributions=params, scoring="f1_weighted", n_iter=30, n_jobs=-1, random_state=42, cv=ps, verbose=self.verbose)
        
        # Note: You need to combine the training and validation set for fitting
        X_combined = pd.concat([self.X_train, self.X_val], axis=0 , ignore_index=True)
        y_combined =  pd.concat([self.y_train, self.y_val], axis=0, ignore_index=True)
        searcher.fit(X_combined, y_combined) 
        
        # Setting the best estimator as the final model
        self.result = searcher.best_estimator_
        
        # Informative print statement
        print(f"Best {'pipeline' if use_pipeline else 'model'} params used:", searcher.best_params_)
   
    
    def evaluate_model(self, plot=True):
        """
        Evaluate the best trained model or pipeline.
        """
        if self.result is None:
            raise ValueError("Model has not been trained yet!")
        
        y_pred = self.result.predict(self.X_test)
        self.report(y_pred, plot)

    def report(self, y_pred, plot=True):
        """
        Print classification report and display confusion matrix.
        """
        class_report = classification_report(self.y_test, y_pred)
        print(class_report)
        
        conf_matrix = confusion_matrix(self.y_test, y_pred) # normalize="true" gives the probortions!!!!
        print(conf_matrix)

        if plot:
            # Infer unique labels from y_test, y_test has to be dataframe
            labels = sorted(list(set(self.y_test)))
            label_strings = [str(label) for label in labels]

            df_cm = pd.DataFrame(conf_matrix, label_strings, label_strings)
            # Naming the conf_matrix df_cm=pd.DataFrame(conf_matrix, ["Sell 0", "Buy 1"],  ["Sell 0", "Buy 1"])
            sns.heatmap(df_cm, fmt='d',annot=True).set(xlabel="Assigned Class", ylabel="True Class", title="Confusion Matrix")
            plt.show()
            ## Add plot_curves_roc directly
            self.plot_curves_roc()
            
    def plot_curves_roc(self):
        """
        Plot ROC and Precision-Recall curves.
        """
        y_pred_prob = self.result.predict_proba(self.X_test)
        
        # Setting up subplots
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

        # ROC Curve and AUC score
        fpr, tpr, thresholds = roc_curve(self.y_test, y_pred_prob[:, 1]) #Only works for binary classification here
        roc_auc = auc(fpr, tpr)
        ax1.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
        ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        ax1.set_xlabel('False Positive Rate')
        ax1.set_ylabel('True Positive Rate')
        ax1.set_title('Receiver Operating Characteristic (ROC) Curve')
        ax1.legend()

        # Precision-Recall Curve
        precision, recall, _ = precision_recall_curve(self.y_test, y_pred_prob[:, 1])
        avg_precision = average_precision_score(self.y_test, y_pred_prob[:, 1])
        ax2.plot(recall, precision, color='darkorange', lw=2, label='Precision-Recall curve (area = %0.2f)' % avg_precision)
        ax2.set_xlabel('Recall')
        ax2.set_ylabel('Precision')
        ax2.set_title('Precision-Recall Curve')
        ax2.legend()

        plt.tight_layout()
        plt.show()

    def plot_learning_curve(self, step_size=0.1):
        """
        Plot the learning curve for the model.
        step_size: Fraction of the maximum size of the training set that will be used in iterations.
                   e.g., step_size=0.1 will use 10%, 20%, ..., 100% of the training data.
        """
        train_sizes, train_scores, test_scores = learning_curve(
            estimator=self.result, 
            X=self.X_train, 
            y=self.y_train, 
            cv=self.kFold,
            train_sizes=np.linspace(0.1, 1.0, int(1/step_size)), 
            scoring="f1_weighted",
            n_jobs=-1
        )

        train_scores_mean = np.mean(train_scores, axis=1)
        train_scores_std = np.std(train_scores, axis=1)
        test_scores_mean = np.mean(test_scores, axis=1)
        test_scores_std = np.std(test_scores, axis=1)

        plt.figure(figsize=(10, 6))
        plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
        plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
        plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="Training score")
        plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="Cross-validation score")
        plt.xlabel("Training examples")
        plt.ylabel("F1 Score")
        plt.legend(loc="best")
        plt.grid(True)
        plt.title("Learning Curve")
        plt.show()


    def selected_features(self):
            """
            Display the features selected by the feature selection step in the pipeline. Ensure that in the pipeline this step is named 'feature_selection'.
            """
            # Get the feature selector from the pipeline
            feature_selector = self.result.named_steps["feature_selection"] #self.result.best_estimator_.named_steps["feature_selection"]
            # Get the mask of selected features
            selected_mask = feature_selector.get_support()
            # Get the feature names from the original dataset
            feature_names = self.X_train.columns
            # Print the selected feature names
            selected_features = feature_names[selected_mask]
            print("Selected features:", selected_features)

    def pca_components(self):
        """
        Display the principal components and their relationship to the original features 
        if using PCA, or the eigenvalues if using KernelPCA. Ensure that in the pipeline this step is named 'feature_selection'.
        """
        # Get the PCA or KernelPCA transformer from the pipeline
        transformer = self.result.named_steps["feature_selection"] #self.result.best_estimator_.named_steps["feature_selection"]
        if isinstance(transformer, PCA):
            # For PCA
            # Get the components
            components = transformer.components_
            # Get the feature names from the original dataset
            feature_names = self.X_train.columns
            # Print the components and their relationship to the original features
            for i, component in enumerate(components):
                print(f"Principal Component {i+1}:")
                for feature, weight in zip(feature_names, component):
                    print(f"{feature}: {weight:.4f}")
                print("\n")

        elif isinstance(transformer, KernelPCA):
            # For KernelPCA
            # Get the eigenvalues
            eigenvalues = transformer.eigenvalues_

            # Print the eigenvalues
            for i, value in enumerate(eigenvalues):
                print(f"Eigenvalue for Kernel Principal Component {i+1}: {value:.4f}")
            print("\n")
        else:
            print("The feature_selection step is neither PCA nor KernelPCA.")

    def feature_importance_plot_shap(self, number_of_sample=100):
        """
        Generate a feature importance plot using SHAP values. Ensure the model's step in the pipeline is named 'model'.
        """
        # Define a wrapper to bypass potential issues
        def model_predict(data):
            return model.predict(data)

        X_test_sample = shap.sample(self.X_test, number_of_sample)  # sampling since computationaly extensive...
        #If you want to see how your model is making predictions on unseen/new data and which features are influencing these decisions, then you should compute the SHAP values on the test data!

        # Extract model from the pipeline
        if hasattr(self.result, 'steps'): 
            model = dict(self.result.steps).get('model', None) 
        else:
            model = self.result 

        # Check model type to determine the appropriate SHAP explainer
        if isinstance(model, (RandomForestClassifier, DecisionTreeClassifier, ExtraTreesClassifier)):
            explainer = shap.TreeExplainer(model)
            shap_values = explainer.shap_values(X_test_sample)

        elif isinstance(model, LogisticRegression):
            explainer = shap.LinearExplainer(model, X_test_sample, feature_dependence="independent") #shap.sample(self.X_train, number_of_sample)?? Some how it does not work!!!
            shap_values = explainer.shap_values(X_test_sample)
        

        elif isinstance(model, SVC):
            explainer = shap.KernelExplainer(model_predict, X_test_sample) ### on X_test
            shap_values = explainer.shap_values(X_test_sample, nsamples="auto", n_jobs=-1)

        elif isinstance(model, ( XGBClassifier, CatBoostClassifier)):
            # Have to use KernelExpaliner since XGB, CatBoost are using categorical data, so TreeExpaliner wont work
            print("Uses KernelExplainer for XGB and CatBoost -- this is very slow!")
            explainer = shap.KernelExplainer(model_predict, X_test_sample) ## on X_test
            shap_values = explainer.shap_values(X_test_sample, nsamples="auto", n_jobs=-1)

        else:
            print("Uses KernelExplainer which is very slow....")
            explainer = shap.KernelExplainer(model_predict, X_test_sample) ## on X_test
            shap_values = explainer.shap_values(X_test_sample, nsamples="auto", n_jobs=-1)

        
        # SHAP DecisionPlot, only apply this on Binary Classification Tasks!
        #For many tree-based classifiers (like RandomForestClassifier), even in binary classification tasks,
        #SHAP provides values for both the classes, resulting in a three-dimensional output: [number_of_classes, number_of_samples, number_of_features]. 
        if len(np.shape(shap_values)) > 2:  # Multi-class classification
            class_index = 1  # Always consider the SHAP values for the positive class in binary classification, poitive class in our case is "buy=1"
            shap_values_selected = shap_values[class_index]
            expected_value_selected = explainer.expected_value[class_index]
            print("-----Here I did something in this loop-----")
        else:
            shap_values_selected = shap_values
            expected_value_selected = explainer.expected_value

        # SHAP SummaryPlot
        shap.summary_plot(shap_values_selected, X_test_sample, feature_names=self.X_train.columns) 

        # SHAP DecisionPlot    
        shap.decision_plot(expected_value_selected, shap_values_selected, feature_names=self.X_train.columns.tolist())

        
        # Different Shap Dependence Plots
        # Get the top features based on the absolute mean SHAP value
        top_features_indices = np.argsort(np.abs(shap_values_selected).mean(0))[-10:]
        # Fetch the corresponding feature names
        top_features_names = self.X_train.columns.to_numpy()[top_features_indices]

        # For each of these top features, find the index in the original dataset
        for feature in top_features_names:
            shap.dependence_plot(feature, shap_values_selected, X_test_sample)


        # Create SVG for a SHAP force plot for a single observation ! STILL UNDER CONSTRUCTION !!!!!
        def shap_plot(j, explainer_Model, shap_values_Model):
            p = shap.force_plot(explainer_Model.expected_value, shap_values_Model[j], X_test_sample.iloc[[j]], matplotlib = True, show = False)
            plt.savefig('shap_figure.svg')
            plt.close()
            return(p)

        #shap_plot(1, explainer, shap_values)

    def permutation_importance_plot(self):
        """
        Generate a feature importance plot using permutation importance.
        """
        # Calculate permutation importances
        result = permutation_importance(self.result, self.X_test, self.y_test, n_repeats=30, random_state=42, n_jobs=-1) 
        sorted_idx = result.importances_mean.argsort()

        # Plotting
        plt.figure(figsize=(10, len(self.X_train.columns) * 0.5))
        plt.boxplot(result.importances[sorted_idx].T, vert=False, widths=1.9)
        plt.yticks(np.arange(1, len(self.X_train.columns) + 1), np.array(self.X_train.columns)[sorted_idx])
        plt.xlabel("Permutation Importance")
        plt.title("Feature Importances via Permutation")
        plt.show()

        # Print Permutation Importance
        feature_importances = pd.DataFrame({'Feature': self.X_train.columns, 'Importance': result.importances_mean})
        feature_importances = feature_importances.sort_values(by='Importance', ascending=False) # Sort the features based on importance
        print(feature_importances)

    def linear_model_coefficients(self):
        """
        Display coefficients for linear models. Ensure the model's step in the pipeline is named 'model'.
        """
        # Check if the model is wrapped inside a pipeline
        if hasattr(self.result, 'steps'): 
            model = dict(self.result.steps).get('model', None) 
        else:
            model = self.result 

        if isinstance(model, (LogisticRegression, LinearRegression)):
            coeffs = model.coef_
            feature_names = self.X_train.columns
            print("linear model coefficients:")
            # Display the coefficients
            for feature, coeff in zip(feature_names, coeffs.ravel()):
                print(f"{feature}: {coeff:.4f}")
        else:
            print("The model is not a linear model.")


    def tree_based_importance(self):
        """
        Display feature importances for tree-based models. Ensure the model's step in the pipeline is named 'model'.
        """
        # Check if the model is wrapped inside a pipeline
        if hasattr(self.result, 'steps'): 
            model = dict(self.result.steps).get('model', None) 
        else:
            model = self.result 

        # Check if the model has the 'feature_importances_' attribute
        if hasattr(model, 'feature_importances_'):
            importances = model.feature_importances_
            feature_names = self.X_train.columns
            sorted_idx = np.argsort(importances)[::-1]
            
            plt.figure(figsize=(10, len(self.X_train.columns) * 0.5))
            # Plot the features in reverse order for correct display
            plt.barh(np.array(feature_names)[sorted_idx][::-1], importances[sorted_idx][::-1])

            # Adding feature importance values next to the bars
            for idx, value in enumerate(importances[sorted_idx][::-1]):
                plt.text(value, idx, f"{value:.4f}", va='center')

            plt.xlabel("Feature Importance")
            plt.title("Feature Importances from Tree-based Model")
            plt.show()
        else:
            print("The model doesn't have a feature_importances_ attribute.")


    def correlation_importance(self):
        """
        Display feature importance based on correlation with the target.
        """
        # Compute correlations of features with the target
        X_to_use=self.X_train.select_dtypes(include='category')
        correlations = X_to_use.join(self.y_train).corr()[self.y_train.name]
        
        # Drop the target itself from the correlations
        correlations = correlations.drop(self.y_train.name, errors='ignore')
        
        # Sort indices by absolute correlation value in descending order
        sorted_idx = correlations.abs().sort_values(ascending=False).index
        
        plt.figure(figsize=(10, len(X_to_use.columns) * 0.5))
        # Plot the features in reverse order for correct display
        plt.barh(sorted_idx[::-1], correlations[sorted_idx][::-1])

        # Adding correlation values next to the bars
        for idx, value in enumerate(correlations[sorted_idx][::-1]):
            plt.text(value, idx, f"{value:.4f}", va='center')

        plt.xlabel("Correlation with Target")
        plt.title("Feature Importances based on Correlation")
        plt.show()


    def analyse_feature_importance_all(self, number_of_sample=100):
        self.feature_importance_plot_shap(number_of_sample)
        self.permutation_importance_plot()
        self.linear_model_coefficients()
        self.tree_based_importance()
        self.correlation_importance()


## Performance Metrics

The F1 score is a measure of a model's accuracy that considers both precision and recall. It's the harmonic mean of these two metrics. Precision refers to the percentage of your results which are relevant, while recall refers to the percentage of total relevant results correctly classified by your algorithm.

The "weighted" F1 score means that each class's F1 score is weighted by the number of true instances for that class. This is useful in multi-class classification problems where you have an imbalanced dataset. That is, the number of instances of each class varies greatly.

In [ ]:
def get_results_cv(func, X_test, y_test):
  std_best_score = func.cv_results_["std_test_score"][func.best_index_]
  print(f"Best parameters: {func.best_params_}")
  print(f"Mean CV score: {func.best_score_: .6f}")
  print(f"Standard deviation of CV score: {std_best_score: .6f}")
  print("Test Score: {:.6f}".format(func.score(X_test, y_test)))

def final_report(y_true, y_pred):
  class_report = metrics.classification_report(y_true, y_pred)
  print(class_report)
  cm = confusion_matrix(y_true, y_pred) ## normalize = "true" gives the proportions!!!!!
  cm = pd.DataFrame(cm, ["Sell",  "Buy"], ["Sell", "Buy"])
  plt.figure(figsize = (10,5))
  sns.heatmap(cm, annot = True, fmt='d', cmap = "Blues").set(xlabel = "Assigned Class", ylabel = "True Class", title = "Confusion Matrix") #fmt = ".2%", 
  plt.show()
  
def plot_curves_roc(model, X_test, y_test):
    # Predicting probabilities
    y_pred_prob = model.predict_proba(X_test)[:, 1]
        
    # Setting up subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # ROC Curve and AUC score
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
    roc_auc = auc(fpr, tpr)
    ax1.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    ax1.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('Receiver Operating Characteristic (ROC) Curve')
    ax1.legend()

    # Precision-Recall Curve
    precision, recall, _ = precision_recall_curve(y_test, y_pred_prob)
    avg_precision = average_precision_score(y_test, y_pred_prob)
    ax2.plot(recall, precision, color='darkorange', lw=2, label='Precision-Recall curve (area = %0.2f)' % avg_precision)
    ax2.set_xlabel('Recall')
    ax2.set_ylabel('Precision')
    ax2.set_title('Precision-Recall Curve')
    ax2.legend()

    plt.tight_layout()
    plt.show()


# Feature Selection

We implemented the following different feature selections functions:

- Random Forest
- Xg Boost
- PCA

### Why Feature Selection?

Feature selection is the process of selecting a subset of relevant features (variables or predictors). It is a critical step that can have a profound impact on the performance of your model. The main benefits of feature selection include:

- Simpler Models
- Less Overfitting
- Speed up Training
- Improved Accuranccy
- Reduces Noise
- Prevents Multicolinearity


It's also important to remember that feature importance doesn't necessarily imply causality; a feature may be important in the context of a particular model without being a causal factor for the outcome being predicted.

In [ ]:
def get_significant_features(X_train, X_test, y_train, n):
    # Feature selection using Extra Trees Classifier on the resampled training data
    model = ExtraTreeClassifier(random_state=42)
    model.fit(X_train, y_train)
    importances = model.feature_importances_
    importances_normalized = np.std([tree.feature_importances_ for tree in
                                        model.estimators_],
                                        axis = 0)

    # Select top features with highest importance scores
    top_features = pd.Series(importances, index=X_train.columns).nlargest(n)

    # Subset X_resampled and X_test with selected features
    X_train_selected = X_train[top_features.index]
    X_test_selected = X_test[top_features.index]

    return X_train_selected, X_test_selected, importances_normalized


def random_forest_feature_selection(X_train, X_test, y_train, n):
    
    model = SelectFromModel(RandomForestClassifier(n_estimators = n))
    model.fit(X_train, y_train)
    
    list_train_rf= X_train.columns[(model.get_support())]
    list_test_rf= X_test.columns[(model.get_support())]

    X_train_rf = X_train[list_train_rf]
    X_test_rf = X_test[list_test_rf]
    
    return X_train_rf, X_test_rf

def xg_boost_feature_selection(X_train, X_test, y_train):
    
    params = { "objective": "multi:softmax", 'num_class': 2 , 'random_state': 42 }
    model= xgb.XGBClassifier(**params)
    select_xgbc = SelectFromModel(estimator = model, threshold = "median")
    select_xgbc.fit(X_train, y_train)

    list_train_xgbc= X_train.columns[(select_xgbc.get_support())]
    list_test_xgbc= X_test.columns[(select_xgbc.get_support())]


    X_train_xgbc = X_train[list_train_xgbc]
    X_test_xgbc = X_test[list_test_xgbc]
    
    return X_train_xgbc, X_test_xgbc

def pca_feature_selection(X_train, X_test, y_train):

    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Create a PCA object and fit it to the scaled data
    pca = PCA(n_components=3) # Select the number of components you want to keep
    pca.fit(X_train_scaled)

    # Transform the data to the selected number of components
    X_train_pca = pca.transform(X_train_scaled)
    X_test_pca = pca.transform(X_test_scaled)

    return X_train_pca, X_test_pca

# Naive Classifiaction/ Baseline

In [ ]:
## Print proportions of each class in cleaned data set
X_combined = pd.concat([X_train, X_val], axis=0, ignore_index=True)
y_combined =  pd.concat([y_train, y_val], axis =0, ignore_index=True)

sell_prop_trainc = sum(y_combined == 0)/y_combined.shape[0]
buy_prop_trainc = sum(y_combined == 1)/y_combined.shape[0]

print(f"Sell proportion:  {sell_prop_trainc: .6f}")
print(f"Buy proportion:  {buy_prop_trainc: .6f}")


############## Naive Classification/ Baseline ##############
priors = [sell_prop_trainc,  buy_prop_trainc]
np.random.seed(42) # set seed

# randomly choose classes 0, 1, with probabilities based on proportions from X_train_cleaned
y_pred = np.random.choice([0, 1], size = len(y_test), replace = True, p = priors) 

f1_weighted = metrics.f1_score(y_test, y_pred, average = "weighted")
print(f"Test Score:  {f1_weighted: .6f}")
final_report(y_test, y_pred)

# Methods




## QDA

In [ ]:
#from sklearn.qda import QDA
# QDA-Modell erstellen und anpassen
qda = QuadraticDiscriminantAnalysis(priors=priors)
qda.fit(X_combined, y_combined)

# Vorhersagen auf Testdaten machen
y_pred = qda.predict(X_test)

final_report(y_test, y_pred)
plot_curves_roc( qda ,X_test, y_test)

## Basic Logistic Regression without Parameter Tuning

In [ ]:
model = LogisticRegression( random_state=42, max_iter= 10000, n_jobs=-1) 
## Default Parameters set as:
#LogisticRegression(penalty='l2', C=1.0,  class_weight=None,  solver='lbfgs', max_iter=100)

pipeline_params = {'ros': [None ], # smote_nc for categorical data
              'scaler': [scaler, mms], # have to scale????
            } 

pipe = imbpipeline(steps=[["scaler", scaler], ["ros", ros], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, model_params=None, pipeline_params=pipeline_params)
train.train_with_crossvalidation(use_pipeline=True, use_grid_search=False) 
train.evaluate_model()
#train.analyse_feature_importance_all(number_of_sample=100)


## Logistic Regression

In [ ]:
model = LogisticRegression( random_state=42, max_iter= 10000, n_jobs=-1) 


pipeline_params = {'ros': [None, ros, smote, borderline_smote , smote_nc], # smote_nc for categorical data
              'scaler': [scaler, mms], # have to scale????
              'model__solver': ['saga'], # 'lbfgs', 'newton-cholesky'
              'model__penalty': ['l2',  None], # Removed None since 'l2' is the default and None would be redundant, 'l1'
              'model__class_weight': ['balanced', None],
              'model__C': [ 0.01, 0.1, 1, 10, 100],
            } 

pipe = imbpipeline(steps=[["scaler", scaler], ["ros", ros], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val,  model=model, pipeline =pipe, model_params=None, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True, use_grid_search=False) 
train.evaluate_model()
#train.plot_learning_curve()
#train.analyse_feature_importance_all(number_of_sample=100) #shap wont work!!!!
#train.permutation_importance_plot()
#train.linear_model_coefficients()
#train.correlation_importance()



## Decision-Tree

In [ ]:
# Initializing decision tree
model = DecisionTreeClassifier(random_state = 42)
#Defult Parameters:
#DecisionTreeClassifier( criterion='gini', splitter='best', max_depth=None,  max_features=None, class_weight=None,)

# hyperparameter to be tested
pipeline_params = {
    'ros': [None, ros, smote, borderline_smote, smote_nc],
    "model__criterion": ['gini', 'entropy'],
    "model__splitter": ['best', 'random'],
    "model__max_features": [None, "auto", "sqrt", "log2"],
    "model__class_weight": [None, "balanced", "balanced_subsample"],
}

pipe = imbpipeline(steps=[["ros", ros], ["model", model]]) 

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_crossvalidation(use_pipeline=True,  use_grid_search=False)  
train.evaluate_model()
#train.plot_learning_curve()
train.analyse_feature_importance_all(number_of_sample=100)


# Plot tree
clf_temp = DecisionTreeClassifier(max_depth=4, random_state = 42)
clf_temp = clf_temp.fit(X_train, y_train)
plt.figure(figsize=(40, 23))
plot_tree(clf_temp, filled=True, feature_names = list(X_train.columns), rounded=True, class_names=["sell", "buy"])
plt.show()



## ExtraTree

In [ ]:
categorical_features = X_train.select_dtypes(include='category').columns.tolist()
print(categorical_features)


model = ExtraTreesClassifier(random_state = 42, n_jobs=-1) #####TreeSSS
## Defult Parameter:
# ExtraTreesClassifier(n_estimators=100, criterion='gini', max_depth=None, max_features='sqrt', class_weight=None,  max_samples=None)

pipeline_params = {
    'ros': [None, ros, smote, borderline_smote, smote_nc],
    "model__criterion": ["gini", "entropy"],
    "model__max_features": [ None, "sqrt", "log2"],
    "model__n_estimators": [50, 100, 200, 250],
    "model__class_weight": [None, "balanced", "balanced_subsample"],
}

pipe = imbpipeline(steps=[["ros", ros], ["model", model]]) 

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_crossvalidation(use_pipeline=True,  use_grid_search=False) 
train.evaluate_model()
#train.plot_learning_curve()
#train.analyse_feature_importance_all(number_of_sample=100)




## Random Forest

In [ ]:
model = RandomForestClassifier(random_state = 42, n_jobs=-1)


pipeline_params = {
    'ros': [None, ros, smote, borderline_smote, smote_nc],
    "model__criterion": ["gini", "entropy"],
    "model__max_features": [ None, "sqrt", "log2"],
    "model__n_estimators": [ 50, 100, 200, 250],
    "model__class_weight": [None, "balanced", "balanced_subsample"],
}


pipe = imbpipeline(steps=[["ros", ros], ["model", model]]) 

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_crossvalidation(use_pipeline=True,  use_grid_search=True) 
train.evaluate_model()
#train.plot_learning_curve()
train.analyse_feature_importance_all(number_of_sample=100)

## Defult Parameters
# RandomForestClassifier(n_estimators=100, criterion='gini', max_depth=None, max_features='sqrt',  class_weight=None,  max_samples=None)



## Stochastic Gradient Desecent

In [ ]:
model = SGDClassifier(random_state = 42, max_iter = 1000, n_jobs =-1)
## Defult Parameters
#SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, l1_ratio=0.15,  max_iter=1000,  learning_rate='optimal',   class_weight=None,)

pipeline_params = {
    'ros': [None, ros, smote, borderline_smote, smote_nc],
    'scaler': [scaler, None, mms], 
    "model__loss": [ 'log_loss', 'modified_huber'], #'hinge',
    "model__penalty": ['l2', 'l1', 'elasticnet'],
}


pipe = imbpipeline(steps=[["ros", ros], ["scaler", scaler], ["model", model]]) 

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True,  use_grid_search=False) 
train.evaluate_model()
#train.plot_learning_curve()
#train.analyse_feature_importance_all(number_of_sample=100)



## AdaBoost

In [ ]:
model = AdaBoostClassifier(random_state=42)
#Deafult Parameters
#AdaBoostClassifier(estimator=None,  n_estimators=50, learning_rate=1.0, algorithm='SAMME.R',  base_estimator='deprecated')

pipeline_params = {
    'ros': [None, ros, smote, borderline_smote, smote_nc],
    'scaler': [scaler, None, mms], 
    "model__learning_rate": [0.01, 0.05, 0.1, 0.3],
    "model__n_estimators": [50, 100, 200, 250],
    "model__algorithm": ['SAMME', 'SAMME.R'],
}


pipe = imbpipeline(steps=[["ros", ros], ["scaler", scaler],["model", model]]) 

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True, use_grid_search=False)  
train.evaluate_model()
#train.plot_learning_curve()
#train.analyse_feature_importance_all(number_of_sample=100)



## XGBoost

In [ ]:
model = xgb.XGBClassifier(objective = "binary:logistic",  enable_categorical=True, random_state = 42, n_jobs =-1) 

pipeline_params = {
    'ros': [None, ros, smote, borderline_smote, smote_nc], # 
    "model__booster": ['gbtree', 'dart'], #'gblinear'
    "model__max_depth": [3, 6, 10 ],
    "model__learning_rate": [ 0.1, 0.3],
}


pipe = imbpipeline(steps=[["ros", ros], ["model", model]]) 

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_crossvalidation(use_pipeline=True, use_grid_search=True) 
train.evaluate_model()
#train.plot_learning_curve()
train.analyse_feature_importance_all(number_of_sample=100)



## CatBoost

In [ ]:
categorical_features = X_train.select_dtypes(include='category').columns.tolist()
print(categorical_features)

X_train_to_use = X_train.copy(deep=True)
X_test_to_use = X_test.copy(deep=True)
X_val_to_use = X_val.copy(deep=True)

## Convert to int or string
for cat in categorical_features:
        X_train_to_use[cat] = X_train_to_use[cat].astype('string')
        X_test_to_use[cat] = X_test_to_use[cat].astype('string')
        X_val_to_use[cat] = X_val_to_use[cat].astype('string')  


model= CatBoostClassifier(cat_features=categorical_features, loss_function="Logloss", random_state = 42, verbose= 500)


pipeline_params = {'ros': [ros, smote_nc, None], # smote, borderline_smote,
                   'model__learning_rate': [0.05, 0.1],
                   'model__depth': [10, 15, 20],
                   'model__iterations': [1000]
}



pipe = imbpipeline(steps=[ ["ros", ros], ["model", model]])

train = TrainModel(X_train_to_use, y_train, X_test_to_use, y_test, X_val_to_use, y_val, model=model, pipeline =pipe, model_params=None, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True,  use_grid_search=False) 
train.evaluate_model()
#train.plot_learning_curve()
train.analyse_feature_importance_all(number_of_sample=100)



## SVM

In [ ]:
model= SVC(random_state=42, probability=True, max_iter= -1) #probability=True enables probabiliry estimates.
## Default Parameters
#SVC(C=1.0, kernel='rbf', degree=3, gamma='scale',  cache_size=200, class_weight=None,  max_iter=-1)

pipeline_params = {
    'ros': [None, ros , smote, borderline_smote, smote_nc], 
    'scaler': [scaler, mms],
    "model__kernel": ["rbf", "sigmoid"], #"linear"
    'model__gamma': ["auto", "scale"],
    'model__class_weight': [None, 'balanced'], 
    'model__C': [ 0.05, 1.0, 5.0, 10.0, 20.0 , 50.0],
}

pipe = imbpipeline(steps=[["ros", ros],["scaler", scaler], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_crossvalidation(use_pipeline=True,  use_grid_search=False)  
train.evaluate_model()
#train.plot_learning_curve()
train.analyse_feature_importance_all(number_of_sample=100)



## NaiveBayes

In [ ]:
model = GaussianNB()

y_combined =  pd.concat([y_train, y_val], axis =0).reset_index(drop=True)
priors = (sum(y_combined == 0)/y_combined.shape[0], sum(y_combined == 1)/y_combined.shape[0])

pipeline_params = {
    'ros': [None, ros, smote, borderline_smote, smote_nc],
    "scaler": [scaler, mms, None],
    "model__priors": [None, priors]
}

pipe = imbpipeline(steps=[["scaler", scaler], ["ros", ros], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True,  use_grid_search=False) 
train.evaluate_model()
#train.plot_learning_curve()
#train.analyse_feature_importance_all(number_of_sample=100)



## MultinomialNB

In [ ]:
model = MultinomialNB()

y_combined =  pd.concat([y_train, y_val], axis =0, ignore_index=True)
priors = (sum(y_combined == 0)/y_combined.shape[0], sum(y_combined == 1)/y_combined.shape[0])

pipeline_params = {
    'ros': [None, ros, smote, borderline_smote, smote_nc],
    "scaler": [scaler, mms, None],
    "model__class_prior": [None, priors],
    "model__force_alpha": [ True, False]
}

pipe = imbpipeline(steps=[["scaler", scaler], ["ros", ros], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True, use_grid_search=False) 
train.evaluate_model()
#train.plot_learning_curve()
#train.analyse_feature_importance_all(number_of_sample=100)



## H20 Package Missing Data Forest

In [ ]:
import h2o
from h2o.estimators import H2ORandomForestEstimator

from h2o.transforms.decomposition import H2OPrincipalComponentAnalysisEstimator
from h2o.estimators import H2OGradientBoostingEstimator


def initialize_h2o():
    h2o.init()

def prepare_data(df, X):
    df_h2o = h2o.H2OFrame(df)
    response = 'Target'
    df_h2o[response] = df_h2o[response].asfactor()
    predictors = X.columns.tolist()
    train, valid, test = df_h2o.split_frame(ratios=[.8, .1], seed=42) #TODO can you do this split so its comparabal to all other methods in the class TrainModel 
    return train, valid, test, predictors, response

def train_model(train, valid, predictors, response):
    model = H2ORandomForestEstimator(ntrees=50, max_depth=20, nfolds=15, binomial_double_trees=False) # binomial_double_trees=True is not supported for SHAP!!!!
    model.train(x=predictors, y=response, training_frame=train, validation_frame=valid)
    return model

def plot_performance_curves(performance):
    fpr = performance.fprs
    tpr = performance.tprs
    roc_auc = performance.auc()
    precision = performance.precision()[0]
    recall = performance.recall()[0]

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    ax1.plot(fpr, tpr, color='b', label=f'AUC = {roc_auc:.2f}')
    ax1.plot([0, 1], [0, 1], color='gray', linestyle='--')
    ax1.set_xlim([0.0, 1.0])
    ax1.set_ylim([0.0, 1.05])
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate')
    ax1.set_title('ROC Curve')
    ax1.legend(loc="lower right")

    ax2.plot(recall, precision, color='b')
    ax2.set_xlim([0.0, 1.0])
    ax2.set_ylim([0.0, 1.05])
    ax2.set_xlabel('Recall')
    ax2.set_ylabel('Precision')
    ax2.set_title('Precision-Recall Curve')

    plt.tight_layout()
    plt.show()

def plot_confusion_matrix(performance):
    confusion_matrix_h2o = performance.confusion_matrix()
    cm_data = pd.DataFrame(confusion_matrix_h2o.to_list())
    cm_data = cm_data.iloc[:2, :2]

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm_data, annot=True, fmt=".0f", cmap="Blues")
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

def plot_feature_importance(model):
    plt.figure(figsize=(10, 8))
    importance = model.varimp(use_pandas=True)
    sns.barplot(x='scaled_importance', y='variable', data=importance)
    plt.title('Feature Importance')
    plt.show()


def random_forest_classification_h2o(df, X, y):
    initialize_h2o()

    train, valid, test, predictors, response = prepare_data(df, X)
    model = train_model(train, valid, predictors, response)
    performance = model.model_performance(test_data=test)
    print(performance)

    plot_performance_curves(performance)
    plot_confusion_matrix(performance)
    plot_feature_importance(model)

    # Use the built-in H2O function to plot feature importance
    model.varimp_plot()


    predictions = model.predict(test)
    h2o.shutdown(prompt=False)



random_forest_classification_h2o(mydata.data, mydata.X, mydata.y)


### TODO apply SHAP Feature Importance for H20 Packages see the function shap in TrainModel() class

In [ ]:
# For 3D plots, we will need Axes3D
from mpl_toolkits.mplot3d import Axes3D
from PyALE import ale


def plot_pdp(model, data, feature, nbins=20):
    """
    Plot PDP for a specific feature.
    """
    pdp = model.partial_plot(data, cols=[feature], nbins=nbins, plot=True)

def plot_ale(model, data, feature):
    # Predict function for your model
    def predict_fn(X):
        df = h2o.H2OFrame(X)
        pred = model.predict(df).as_data_frame().values
        return pred
    
    # Get numpy data without target
    X = data.drop('Target', axis=1).as_data_frame().values

    fig, ax = plt.subplots()
    ale(predict_fn, X, feature, ax=ax)
    ax.set_title(f"ALE plot for {feature}")
    plt.show()
    
def plot_interaction_heatmap(model, data, feature1, feature2):
    """
    Plot two-way interaction heatmap.
    """
    pdp = model.partial_plot(data, cols=[feature1, feature2], plot=False)
    interaction_data = pdp[0].as_data_frame()
    pivoted_data = interaction_data.pivot(index=feature1, columns=feature2, values='mean_response')
    sns.heatmap(pivoted_data, cmap='YlGnBu', annot=True)
    plt.title(f"Interaction between {feature1} and {feature2}")
    plt.show()

def plot_interaction_3d(model, data, feature1, feature2, feature3):
    """
    Plot three-way interaction in 3D.
    """
    pdp = model.partial_plot(data, cols=[feature1, feature2, feature3],  plot=False)
    interaction_data = pdp[0].as_data_frame()

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(interaction_data[feature1], interaction_data[feature2], interaction_data[feature3], c=interaction_data['mean_response'], cmap='YlGnBu')
    ax.set_xlabel(feature1)
    ax.set_ylabel(feature2)
    ax.set_zlabel(feature3)
    ax.set_title(f"Interaction among {feature1}, {feature2}, and {feature3}")
    plt.show()

# Methods with Feature Selection

## Logisitc Regression

In [ ]:
model = LogisticRegression( random_state=42, max_iter= 10000, n_jobs=-1)  #solver='lbfgs'


pipeline_params = {'ros': [None, ros, smote, borderline_smote, smote_nc], # upsampling or not
              'scaler': [scaler, None, mms], # scaling input by standardizing or min-max scaling or not scaling at all
              'model__solver': ['lbfgs', 'saga'],
              'model__multi_class': ['auto'],
              'model__penalty': ['l2', 'l1', 'elasticnet'], # Removed None since 'l2' is the default and None would be redundant
            } 


pipe = imbpipeline(steps=[["scaler", scaler], ["ros", ros], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, model_params=None, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True, use_grid_search=False) 
train.evaluate_model()


### Logistic Regression with Random Forest Feature Selection

In [ ]:
# Define the classifier and its parameters
model = LogisticRegression(solver='saga', random_state=42, max_iter= 1000, n_jobs=-1)
# Define feature selector and its parameters
feat_sel = RandomForestClassifier(random_state = 42, n_estimators=100, n_jobs=-1)

# Construct the pipeline
pipe = imbpipeline(steps=[["scaler", scaler], ["feature_selection",  SelectFromModel(estimator=feat_sel, threshold="median")],
                          ["ros", ros], ["model", model]])


train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True,  use_grid_search=False) 
train.evaluate_model()
train.selected_features()



### Logistic Regression with XGBoost Feature Selection

In [ ]:
# Define the classifier and its parameters
model = LogisticRegression(solver='saga', random_state=42, max_iter= 1000, n_jobs=-1)
# Define feature selector and its parameters
feat_sel = xgb.XGBClassifier(objective = "binary:logistic", enable_categorical=True, random_state = 42, n_jobs =-1)
# Construct the pipeline
pipe = imbpipeline(steps=[["scaler", scaler], ["feature_selection",  SelectFromModel(estimator=feat_sel, threshold="median")],
                          ["ros", ros], ["model", model]])


train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True,  use_grid_search=False) 
train.evaluate_model()
train.selected_features()


### Logistic Regression with PCA Feature Selection

In [ ]:
# Define the classifier and its parameters
model = LogisticRegression(solver='saga', random_state=42, max_iter= 1000, n_jobs=-1)

pca = KernelPCA(random_state = 42, eigen_solver = "arpack")

pipeline_params = { 
    'feature_selection__n_components': np.arange(5, 10, 1),
    "feature_selection__kernel": ["linear", "sigmoid"],
    'model__penalty': ['l2', 'l1', 'elasticnet'],
    'ros': [None, ros, smote, borderline_smote, smote_nc],
    'scaler': [scaler, mms], #scaling since PCA?
}

# Construct the pipeline
pipe = imbpipeline(steps=[["scaler", scaler], ["feature_selection",  pca],
                          ["ros", ros], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True,  use_grid_search=False)  
train.evaluate_model()
train.pca_components()


 ## Random Forest

 Random Forest is a popular and versatile machine learning method that is capable of performing both regression and classification tasks. It is also used for dimensionality reduction, treats missing values, outlier values, and other things.

 Random Forests generally have a high prediction accuracy and are quite efficient on large datasets.

### Random Forest without Feature Selection

In [ ]:
model = RandomForestClassifier(random_state = 42, n_jobs=-1)

pipeline_params = {
    "model__criterion": ["gini", "entropy"],
    "model__max_features": [ "sqrt", "log2", None],
    "model__n_estimators": np.array([ 50, 100, 200, 250]),
    "model__class_weight": [None, "balanced", "balanced_subsample"],
    "model__max_depth": np.array([None, 5, 10, 20]),
    'ros': [None, ros, smote, borderline_smote, smote_nc],
}

pipe = imbpipeline(steps=[["ros", ros], ["model", model]]) 

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True,  use_grid_search=False) 
train.evaluate_model()

### Random Forest with Random Forest Feature Selection

In [ ]:
feat_sel = RandomForestClassifier(random_state = 42, n_estimators=100, n_jobs=-1)
pipe = imbpipeline(steps=[["feature_selection",  SelectFromModel(estimator = feat_sel, threshold = "median")],
                          ["ros", ros], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True,  use_grid_search=False) 
train.evaluate_model()
train.selected_features()


###  Random Forest with XGBoost Feature Selection

In [ ]:
feat_sel = xgb.XGBClassifier(objective = "binary:logistic", enable_categorical=True, random_state = 42, n_jobs =-1)
pipe = imbpipeline(steps=[["feature_selection",  SelectFromModel(estimator = feat_sel, threshold = "median")],
                          ["ros", ros], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True, use_grid_search=False) 
train.evaluate_model()
train.selected_features()


### Random Forest with PCA Feature Selection

In [ ]:
model = RandomForestClassifier(random_state = 42)

pca = KernelPCA(random_state = 42, eigen_solver = "arpack")

pipeline_params = { "model__criterion": ["gini", "entropy"],
                    "model__max_features": [ "sqrt", "log2"],
                    "model__max_depth": np.array([None, 5, 10, 20]),
                    "model__n_estimators": np.array([ 50, 100, 200, 250]),
                    "model__class_weight": [None, "balanced", "balanced_subsample"],
                    "feature_selection__kernel": ["linear", "sigmoid"],
                    'feature_selection__n_components': np.arange(5, 10, 1),
                    'ros': [None, ros, smote, borderline_smote, smote_nc],
                    'scaler': [ mms, scaler] ### Have to scale for PCA right?
}
    

pipe = imbpipeline(steps=[["scaler", scaler], ["feature_selection", pca ],  ["ros", ros], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True, use_grid_search=False) 
train.evaluate_model()
train.pca_components()


## Xg-Boost

### XgB without Feature Selection

In [ ]:
model = xgb.XGBClassifier(objective ="binary:logistic", enable_categorical=True, random_state = 42, n_jobs =-1)

pipeline_params = {
    "model__booster": ['gbtree', 'dart'], 
    "model__max_depth": [3, 6, 10],
    "model__learning_rate": [ 0.1, 0.3],
    'ros': [None, ros, smote_nc], #smote, borderline_smote, 
}


pipe = imbpipeline(steps=[["ros", ros], ["model", model]]) 

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True,  use_grid_search=False) 
train.evaluate_model()

### Xbg with Random Forest 

In [ ]:
feat_sel = RandomForestClassifier(random_state = 42, n_estimators=100, n_jobs=-1)

pipe = imbpipeline(steps=[["feature_selection",  SelectFromModel(estimator=feat_sel, threshold="median")],
                          ["ros", ros], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True, use_grid_search=False) 
train.evaluate_model()
train.selected_features()


### Xbg with XBg Featrue Selection

In [ ]:
feat_sel = xgb.XGBClassifier(objective ="binary:logistic", enable_categorical=True, random_state = 42, n_jobs =-1)
pipe = imbpipeline(steps=[["feature_selection",  SelectFromModel(estimator = feat_sel, threshold = "median")],
                          ["ros", ros], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val,  model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True, use_grid_search=False) 
train.evaluate_model()
train.selected_features()


### XGB with PCA

In [ ]:
pca = KernelPCA(random_state = 42, eigen_solver = "arpack")
#pca = PCA()
pipeline_params = { "model__booster": ['gbtree', 'dart'], 
    "model__max_depth": [3, 6, 10],
    "model__learning_rate": [ 0.1, 0.3],
    'ros': [None, ros, smote_nc], #smote, borderline_smote,
    'scaler': [ mms, scaler], ### have to scale since PCA right?
    "feature_selection__kernel": ["linear", "sigmoid"],
    'feature_selection__n_components': np.arange(5, 10, 1),  
}
    

pipe = imbpipeline(steps=[["scaler", scaler], ["feature_selection", pca ],  ["ros", ros], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val , y_val,  model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True,  use_grid_search=False) 
train.evaluate_model()
train.pca_components()


## Support Vector Machines

Support Vector Machines (SVMs) aim to find a distinct hyperplane in a high-dimensional space to classify data points. They excel in managing high-dimensional data, even when dimensions outnumber samples, and are memory-efficient by leveraging a subset of training points called "support vectors" in decision-making.

However, SVMs struggle with large feature sets compared to samples, requiring careful kernel and regularization term selection to avoid over-fitting.

### SVM without Feature Selection

In [ ]:
model= SVC(random_state=42, probability=True, max_iter= 1000)

pipeline_params = {
    'ros': [ros, smote, borderline_smote, smote_nc, None], 
    "model__kernel": ["linear", "rbf"],
    'ros': [None, ros, smote, borderline_smote, smote_nc],
    'scaler': [ mms, scaler]
}

pipe = imbpipeline(steps=[["ros", ros], ["scaler", scaler],["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True,  use_grid_search=False) 
train.evaluate_model()


###  SVM with RandomForest Feature Selection

In [ ]:
# Define feature selector and its parameters
model= SVC(random_state=42,probability=True, max_iter= 1000)
feat_sel = RandomForestClassifier(random_state = 42, n_estimators=100, n_jobs=-1)

pipe = imbpipeline(steps=[["feature_selection",  SelectFromModel(estimator=feat_sel, threshold="median")],
                          ["ros", ros], ["scaler", scaler], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True,  use_grid_search=False) 
train.evaluate_model()
train.selected_features()


### SVM with XgBoost Feature Selection

In [ ]:
model= SVC(random_state=42,probability=True, max_iter= 1000)
feat_sel = xgb.XGBClassifier(objective ="binary:logistic", enable_categorical=True, random_state = 42, n_jobs =-1)

pipe = imbpipeline(steps=[["feature_selection", SelectFromModel(estimator=feat_sel, threshold="median")],
                          ["ros", ros],["scaler", scaler], ["model", model]])

train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True, use_grid_search=False) 
train.evaluate_model()
train.selected_features()


### SVM with PCA Feature Selection

In [ ]:
model= SVC(random_state=42, probability=True, max_iter= 1000)
pca = KernelPCA(random_state = 42, eigen_solver = "arpack")


pipeline_params = {
    'ros': [None, ros, smote, borderline_smote, smote_nc],
    'scaler': [ mms, scaler],
    "model__kernel": ["linear", "rbf"],
    "model__C": [ 10],
    "model__gamma": ["scale"],
    "feature_selection__kernel": ["linear", "sigmoid"],
    'feature_selection__n_components': np.arange(5, 10, 1),
}

pipe = imbpipeline(steps=[["scaler", scaler], ["feature_selection", pca], ["ros", ros], ["model", model]])


train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline =pipe, pipeline_params=pipeline_params)
train.train_with_validation(use_pipeline=True, use_grid_search=False) 
train.evaluate_model()
train.pca_components()


# Feature Importance



## PCA-Feature-Importance

- PCA but only for continuous data, if not continuous try Factor Analysis of Mixed Data (FAMD) before applying FAMD one need to normalize the data
- Check Skewness and maybe try Yeo-Johnson transformation before standardisation.? not necessary?

Since n_components is a hyperparameter, it does not learn from the data. We have to manually specify its value (tune the hyperparameter) before we run the PCA() function.
Method 1: If your sole intention of doing PCA is for data visualization, you should select 2 or 3 principal components.
Method 2: If you want an exact amount of variance to be kept in data after applying PCA, specify a float between 0 and 1 to the hyperparameter n_components.
Method 3: Plot the explained variance percentage of individual components and the percentage of total variance captured by all principal components. Method 3 is the best!

In [ ]:
def feature_importance_pca(X_train, scale =True, threshold=0.95):
    # Scale the dataset; This is very important before you apply PCA
    if scale == True:
        sc = StandardScaler()
        sc.fit(X_train)
        X_std = sc.transform(X_train)
    else:
        print("Make sure you don't scale twice")

    # n_components=2 or 3 or try 0.85 for 85% of the variance in the original data, then ask for pca.n_components_ to figure out how many were used
    pca = PCA() #Here, PCA is applied to the scaled data. The number of principal components isn't specified, so by default, it will equal the number of original features.
    pca.fit_transform(X_std)
    #
    # Determine explained variance using explained_variance_ration_ attribute
    exp_var_pca = pca.explained_variance_ratio_
    #
    # Cumulative sum of eigenvalues; This will be used to create step plot
    # for visualizing the variance explained by each principal component.
    #
    cum_sum_eigenvalues = np.cumsum(exp_var_pca)


    plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
    plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, color= 'red', where='mid',label='Cumulative explained variance')

     # Adding a horizontal line at y=0.95 (95.0% threshold)
    if 0 < threshold < 1:
        plt.axhline(threshold,  color='g', linestyle='dashed', label=f'{threshold*100}% threshold')

    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal component index')
    plt.legend(loc='best')
    plt.tight_layout()
    plt.show()


########################
feature_importance_pca(X_train)

In [ ]:
def feature_importance_pca2(model, X_train, y_train, X_test, y_test, scale=True, pca_components=2):
# https://scikit-learn.org/stable/auto_examples/inspection/plot_permutation_importance.html

    if scale == True:
        scaler = StandardScaler()
        scaler.fit(X_train)
        feature_scaled_train = scaler.transform(X_train)
        feature_scaled_test = scaler.transform(X_test)

    else:
        print("Make sure you don't scale twice")
  
    # Perform PCA
    pca = PCA(n_components=pca_components)
    X_train_pc = pca.fit_transform(feature_scaled_train)
    X_test_pc = pca.transform(feature_scaled_test)

    # Train the model
    model.fit(X_train_pc, y_train)
    # Predict using the model
    y_pred = model.predict(X_test_pc)

    # Calculate and print precision and accuracy
    p = precision_score(y_test, y_pred)
    a = accuracy_score(y_test, y_pred)
    print('Precision Score:', p.round(4))
    print('Accuracy Score:', a.round(4))

    # Plot feature importance (assuming the model has the attribute feature_importances_)
    #Mean decrease in impurity (MDI) 
    mdi = pd.DataFrame(model.feature_importances_, columns=['MDI'], index=['pc_{}'.format(i) for i in range(pca.n_components_)]).sort_values('MDI', ascending=False)
    plt.figure(figsize=(10,6))
    plt.bar(mdi.index, mdi.MDI)
    plt.title("Feature Importance: MDI")
    plt.show()

    # Plot PCA loadings
    fig, axs = plt.subplots(pca.n_components_, 1, sharex=True, figsize=(15, 10))
    for i in range(pca.n_components_):
        axs[i].bar(X_train.columns, pca.components_[i])
        axs[i].set_ylabel('component '+str(i)+' loading')
    axs[0].set_title('PCA Loadings')
    axs[-1].set_xlabel('Features')
    fig.tight_layout()

    # Most important PC correlation
    X_train_pc_df = pd.DataFrame(X_train_pc, columns=['pc_{}'.format(i) for i in range(pca.n_components_)], index=X_train.index)
    corr_mi_pc = pd.concat([X_train, X_train_pc_df['pc_1']], axis=1).corr().iloc[:,-1].sort_values().to_frame()
    print(corr_mi_pc)

################################


model = RandomForestClassifier(n_estimators=1000, max_depth=5)
feature_importance_pca2(model, X_train, y_train, X_test, y_test, scale=True, pca_components=4)

## Feature Importance Stuff already in the class TrainModel()

In [ ]:
def feature_importance_plot_simple(model, X_clean , y_clean, normalized=True):

    model.fit(X_clean, y_clean)

    if normalized == True:
        importances = np.std([model.feature_importances_ for model in model.estimators_], axis = 0) ## or MinMaxSclaer or StandardScaler???
    else:
        importances= model.feature_importances_ 


    top_features = pd.Series(importances, index= X_clean.columns).sort_values(ascending=False)
    n = len(top_features)
    
    # Plot feature importance of all features
    plt.figure(figsize=(6, 5))
    plt.bar(range(n), top_features[:n], align='center')

    for i in range(n):
        plt.text(i, top_features[i]+1e-4 , round(top_features[i], 3), ha='center', fontsize = 6)

    plt.xlim([-1, n])
    plt.xticks(range(n), top_features.index[:n], rotation=90)
    plt.xlabel('Features')
    plt.ylabel('Feature Importance')
    plt.tight_layout()

In [ ]:
def combined_importance_plot(model, X_clean, y_clean, normalized=True, threshold=0.95):
    """
    Plots the feature importance and above it, plots the cumulative importance as a step function.
    """
    # Compute feature importances
    model.fit(X_clean, y_clean)
    
    if normalized:
        importances = np.std([tree.feature_importances_ for tree in model.estimators_], axis=0)
    else:
        importances = model.feature_importances_

    sorted_indices = np.argsort(importances)[::-1]
    top_features = pd.Series(importances[sorted_indices], index=X_clean.columns[sorted_indices])
    
    # Compute cumulative importance
    cumulative_importances = np.cumsum(top_features.values)
    
    n = len(top_features)
    
    # Initialize the figure
    fig, ax1 = plt.subplots(figsize=(10, 6))
    
    # Plot feature importances as bars
    ax1.bar(range(n), top_features.values, align='center', alpha=0.8, label='Feature Importance')

    for i in range(n):
        plt.text(i, top_features[i]+1e-4 , round(top_features[i], 3), ha='center', fontsize = 6)

    ax1.set_xlabel('Features')
    ax1.set_ylabel('Feature Importance')
    ax1.set_xticks(range(n))
    ax1.set_xticklabels(top_features.index, rotation=90, fontsize=8)
    
    # Plot cumulative importance as a step function
    ax1.step(range(n), cumulative_importances, where="mid", color="r", label='Cumulative Importance')
    
    # Display the threshold if necessary
    if 0 < threshold < 1:
        ax1.hlines(threshold, xmin=0, xmax=n, colors='g', linestyles='dashed', label=f'{threshold*100}% threshold')
        # Annotate the plot with the number of features needed for the desired cumulative importance
        needed_num_features = np.argmax(cumulative_importances >= threshold) + 1
        ax1.annotate(f'{needed_num_features} features', 
                     xy=(needed_num_features, threshold), 
                     xytext=(needed_num_features, threshold+0.05),
                     arrowprops=dict(facecolor='black', arrowstyle='->'),
                     fontsize=9,
                     color='g')
    
    # Add legend to show both types of importance
    ax1.legend(loc="upper left")
    ax1.set_title(f"{model}")
    plt.tight_layout()
    plt.show()

In [ ]:
#############
tree = ExtraTreesClassifier(random_state=42)
combined_importance_plot(tree, X_train, y_train, normalized=False, threshold=0.95)

forest = RandomForestClassifier(random_state = 42)
combined_importance_plot(forest, X_train, y_train, normalized=False, threshold=0.95)

xgbc = xgb.XGBClassifier(random_state = 42, enable_categorical=True)
combined_importance_plot(xgbc, X_train, y_train, normalized=False, threshold=0.95)

decision = DecisionTreeClassifier(random_state=42)
combined_importance_plot(decision, X_train, y_train, normalized=False, threshold=0.95)

In [ ]:
def catboost_feature_importance(X_train, y_train, X_test, y_test):

    categorical_features = X_train.select_dtypes(include='category').columns.tolist()
    ## Convert to int or string
    for cat in categorical_features:
        X_train[cat] = X_train[cat].astype('string')
        X_test[cat] = X_test[cat].astype('string') 
      
    # Create a CatBoostClassifier
    model = CatBoostClassifier(iterations=1000, 
        learning_rate=0.05, 
        depth=6,
        cat_features=categorical_features,
        random_state = 42,
        verbose=200)
    
    # Fit the classifier on the data
    model.fit(X_train, y_train, cat_features=categorical_features, verbose=0)
    
    # Get the feature importances
    importances = model.feature_importances_
    
    # Create a DataFrame to store the feature importances
    feature_importances = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})
    
    # Sort the features based on importance
    feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

    # Select the top 17 features
    feature_importances = feature_importances.head(17)
    
    # Plot the feature importances
    plt.figure(figsize=(12, 8))  # Adjusted size for better visibility of annotations
    bars = plt.barh(feature_importances['Feature'], feature_importances['Importance'], align='center')
    
    for bar in bars:
        plt.text(bar.get_width() - (0.02 * max(importances)),  # Adjust 0.02 for distance from the end of bar
                 bar.get_y() + bar.get_height()/2,
                 f'{bar.get_width():.2f}',  # Formatting to 2 decimal places
                 ha='center', va='center',
                 color='black')
    
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.title('Feature Importance using CatBoost')
    plt.gca().invert_yaxis()  # This will display the most important feature at the top
    plt.show()


In [ ]:
##############################################
catboost_feature_importance(X_train.copy(deep=True), y_train.copy(deep=True) , X_test.copy(deep=True) , y_test.copy(deep=True))

In [ ]:
def permutation_feature_importance(model, X_train, y_train):

    # Fit the classifier on the data
    model.fit(X_train, y_train)

    # Calculate permutation importances
    results = permutation_importance(model, X_train, y_train, scoring='accuracy', random_state=42)
    # Get the feature importances
    importances = results.importances_mean
    # Create a DataFrame to store the feature importances
    feature_importances = pd.DataFrame({'Feature': X_train.columns, 'Importance': importances})
    # Sort the features based on importance
    feature_importances = feature_importances.sort_values(by='Importance', ascending=False)
    # Print the feature importances
    print(feature_importances)

    # Select the top 17 features
    feature_importances = feature_importances.head(17)
    # Plot the feature importances
    plt.figure(figsize=(12, 8))  # Adjusted size for better visibility of annotations
    bars = plt.barh(feature_importances['Feature'], feature_importances['Importance'], align='center')
    
    for bar in bars:
        plt.text(bar.get_width() - (0.02 * max(importances)),  # Adjust 0.02 for distance from the end of bar
                 bar.get_y() + bar.get_height()/2,
                 f'{bar.get_width():.2f}',  # Formatting to 2 decimal places
                 ha='center', va='center',
                 color='black')
    
    plt.xlabel('Importance')
    plt.ylabel('Feature')
    plt.title('Feature Importance using Permutation Importance for ????? Classificer')
    plt.gca().invert_yaxis()  # This will display the most important feature at the top
    plt.show()


In [ ]:
###################
model = DecisionTreeClassifier(random_state = 42)
permutation_feature_importance(model, X_train.copy(deep=True), y_train.copy(deep=True))

# Random Forest Classifier for each year

### Accuracy with random prediction


In [ ]:
def random_forest_per_year():
  # Initializing dataset path
  path = os.path.join(sys.path[0])
  # Loading and Handling dataset
  mydata_new = OurDataset(path)

  ### Removing and sorting stuff
  mydata_new.remove_timestamps()
  mydata_new.remove_duplicates()
  mydata_new.sort_after_date()
  mydata_new.remove_columns(columns = ['Financial_Ratio_11', 'Financial_Ratio_14'])
  
  years = mydata_new.list_years()
  years.pop(0) ## remove first year, not sufficient data

  for index, year in enumerate(years):
        print("\n\n")
        print(f" -------------{year}--------------- \n")
        year_df = mydata_new.get_year_dataset(year)
        year_df.split_dataset(imputer="iterative", remove_outliers=True) # drop_all will not work here!
        X_train, X_test, X_val, y_train, y_test, y_val = year_df.X_train, year_df.X_test, year_df.X_val, year_df.y_train, year_df.y_test, year_df.y_val

        model = RandomForestClassifier(random_state = 42)
        pipeline_params = {
          "model__criterion": ["gini", "entropy"],
          "model__max_features": [ "sqrt", "log2"],
          "model__max_depth": np.array([None, 5, 10]),
          "model__n_estimators": np.array([ 1, 5, 10, 50, 100]),
          "model__class_weight": [None, "balanced", "balanced_subsample"],
          }
        pipe = imbpipeline(steps=[["ros", ros], ["model", model]])
        train = TrainModel(X_train, y_train, X_test, y_test, X_val, y_val, model=model, pipeline=pipe, pipeline_params=pipeline_params)
        print(f"Random Forrest Classifier trained on data from {year}\n")
        train.train_with_validation()
        train.evaluate_model()
        plt.show()
        print("\n\n")
        

In [ ]:
##############################
random_forest_per_year()

# Neural Network

In [ ]:
import torch
import os
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as T
import glob
from torch import nn
import torch.nn.functional as F
from torchvision import models, transforms
import matplotlib.pyplot as plt

# set random seed to ensure reproducibility
torch.manual_seed(0xbeef)

### Initializing the Dataset

In [ ]:
path = os.path.join(sys.path[0])
mydata_nn = OurDataset(path)

### Removing and sorting stuff
mydata_nn.remove_timestamps()
mydata_nn.remove_duplicates()
mydata_nn.sort_after_date() 



## Imputing Missing Values and Outliers
# if dropping all missing columns be sure to drop the column 'Financial_Ratio_11? since we only got data for 2022 and 2023.
mydata_nn.remove_columns(columns = ['Financial_Ratio_11', 'Financial_Ratio_14']) 
mydata_nn.split_dataset(imputer = "iterative", remove_outliers=True)
X_train, X_test, y_train, y_test = mydata_nn.X_train, mydata_nn.X_test, mydata_nn.y_train.values, mydata_nn.y_test.values

scaler = MinMaxScaler() 
X_train_std = scaler.fit_transform(X_train)
X_test_std = scaler.transform(X_test)

X_train_std, X_val_std, y_train, y_val = train_test_split(X_train_std, y_train, stratify = y_train, test_size = 0.2, random_state = 42) ### Train and Validation Set!

y_test = y_test.to_numpy()
y_val = y_val.to_numpy()
y_train = y_train.to_numpy()

print(type(y_test))
print(type(X_train_std))
print(type(X_test_std))
print(type(X_val_std))
print(type(y_val))
print(type(y_train))



In [ ]:
class CustomDataSet(Dataset):
    def __init__(self, X, y):
        self.X = X.astype('float')
        self.y = y.astype('float')

    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        # provide row `idx` from X and y
        return torch.from_numpy(self.X[idx, :]).type(torch.FloatTensor), torch.tensor(self.y[idx]).type(torch.FloatTensor)        

In [ ]:
trainset = CustomDataSet(X_train_std, y_train)
testset  = CustomDataSet(X_test_std, y_test)
valset = CustomDataSet(X_val_std, y_val)

In [ ]:
batch_size = 128
trainloader = DataLoader(trainset, batch_size = batch_size, shuffle = True)
testloader = DataLoader(testset, batch_size = batch_size, shuffle = False)
valloader= DataLoader(valset, batch_size = batch_size, shuffle = True)

In [ ]:
class CustomNeuralNet(nn.Module):
    def __init__(self, input_dimension):
        super(CustomNeuralNet, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(input_dimension, 100),
            nn.ReLU(),
            nn.Linear(100, 200),
            nn.ReLU(),
            nn.Linear(200, 200),
            nn.ReLU(),
            nn.Linear(200, 200),
            nn.ReLU(),
            nn.Linear(200, 1)
        )
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        output = self.classifier(x)
        return self.sigmoid(output)

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    loss_vals = []
    correct = 0.0
    for _, (X, y) in enumerate(dataloader):
        pred = model(X)
        y = y.type(torch.float).unsqueeze(1)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_vals.append(loss.item())

        # binary classification, round to get the most likely result
        correct += (torch.round(pred) == y).type(torch.float).sum().item()
    return correct/size, sum(loss_vals)/len(loss_vals)

In [ ]:
def validate(dataloader, model, loss_function):
    size = len(dataloader.dataset)
    correct = 0.0
    loss_vals = []
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            y = y.type(torch.float).unsqueeze(1)
            loss_vals.append(loss_function(pred, y).item())
            # binary classification, round to get the most likely result
            correct += (torch.round(pred) == y).type(torch.float).sum().item()
    return correct/size, sum(loss_vals)/len(loss_vals)

In [ ]:
def run_training(model, trainloader, valloader, loss_function, optimizer, epochs=5, model_path='./model.pt'):
    best_accuracy = 0.0
    best_loss = 1.0
    consecutive_worse = 0
    for t in range(epochs):
        print(f"================ Epoch {t} ================\n")
        train_accuracy, train_loss = train(trainloader, model, loss_function, optimizer)
        print(f"Train Accuracy: {(100*train_accuracy):>0.1f}%, Train Loss: {train_loss:>8f} \n")
        val_accuracy, val_loss = validate(valloader, model, loss_function)
        print(f"Validation Accuracy: {(100*val_accuracy):>0.1f}%, Validation Loss: {val_loss:>8f} \n")
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            best_loss = val_loss
            torch.save(model.state_dict(), model_path)
            consecutive_worse = 0
        else:
            consecutive_worse += 1
        if consecutive_worse >= 5:
            print(f"Early stop epoch {t}")
            break
    print(f"Done! Best Model Accuracy: {(100*best_accuracy):>0.1f}, Best Model loss: {best_loss}")
    # Load the best model
    model = CustomNeuralNet(len(X_train_std[0]))
    model.load_state_dict(torch.load(model_path))
    model.eval()

In [ ]:
model = CustomNeuralNet(len(X_train_std[0]))
#print(model)
best_model_path = './custom_nn_model.pt'
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
trained_model = run_training(model, trainloader, valloader, loss_function, optimizer, model_path=best_model_path, epochs=100)

In [ ]:
# run the model on the test data
test_accuracy, _ = validate(testloader, model, loss_function)
print(f"Test Accuracy: {(100*test_accuracy):>0.1f}%\n")
